#Librerías necesarias para la realización del TP

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import math

In [ ]:
!pip install plotly
!pip install pyclustertend
import plotly.express as px

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#no quiero notacion cientifica
pd.options.display.float_format = '{:20,.2f}'.format

In [ ]:
def scatter_predicted_versus_real(pred,real,size=4):
  sns.scatterplot(x=pred, y=real,s=size)

#Datasets necesarios para la realización del TP

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
quick_exec = False # setear a true para prevenir dibujos

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

try:
  # el archivo ya está cargado en el sistema
  csv = pd.read_csv('properati_argentina_2021.csv')
except:
  # cargar usando el acceso directo 'Datasets' 
  # puesto en la carpeta MyDrive del drive del usuario
  csv = pd.read_csv('drive/MyDrive/Datasets/properati_argentina_2021.csv')

#**1. Análisis Exploratorio y Preprocesamiento de Datos**

## Se separa el conjunto de datos inicial en 2: conjunto de entrenamiento (80%) y un conjunto de test (20%).

In [ ]:
from sklearn.model_selection import train_test_split

df_prop_x = csv.drop(columns=['property_price'])
df_prop_y = csv['property_price'].copy()

train, test, train_target, test_target = train_test_split(df_prop_x, df_prop_y, test_size=0.2,random_state=12)

In [ ]:
csv.shape

(460154, 20)

In [ ]:
train.shape

(368123, 19)

In [ ]:
test.shape

(92031, 19)

##Se filtran los datos al subconjunto pedido por el enunciado, es decir, los anuncios de propiedades de tipo vivienda ( Casa, PH y Departamento ) ubicados en Capital Federal cuyo tipo de operación sea venta y su precio se encuentre en dólares (USD).

In [ ]:
def index_caba(df):
  return df.loc[(csv['property_type'].isin(['Casa','PH','Departamento'])) & (csv.operation == 'Venta') & (csv.place_l2 == 'Capital Federal') & (csv.property_currency == 'USD')]

In [ ]:
prop_caba_data = index_caba(train)

In [ ]:
prop_caba_test =  index_caba(test)

Se aplica el filtrado a targets:

In [ ]:
train_target = train_target.filter(prop_caba_data.index)
test_target = test_target.filter(prop_caba_test.index)

##**a) Exploración Inicial :**

Se observa un resumen conciso del DataSet obtenido para train:

In [ ]:
prop_caba_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75517 entries, 100248 to 391107
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        75517 non-null  object 
 1   start_date                75517 non-null  object 
 2   end_date                  75517 non-null  object 
 3   created_on                75517 non-null  object 
 4   latitud                   72553 non-null  float64
 5   longitud                  72553 non-null  float64
 6   place_l2                  75517 non-null  object 
 7   place_l3                  75183 non-null  object 
 8   place_l4                  2917 non-null   object 
 9   place_l5                  0 non-null      object 
 10  place_l6                  0 non-null      float64
 11  operation                 75517 non-null  object 
 12  property_type             75517 non-null  object 
 13  property_rooms            74687 non-null  float64
 14  

In [ ]:
prop_caba_data.head(3)

,id,start_date,end_date,created_on,latitud,longitud,place_l2,place_l3,place_l4,place_l5,place_l6,operation,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_currency,property_title
100248,bQGhCCPR+p49agTNUppQ7Q==,2021-04-07,2021-04-13,2021-04-07,-34.64,-58.46,Capital Federal,Flores,NaN,NaN,NaN,Venta,Casa,7.00,5.00,260.00,260.00,USD,VENTA CASA COCHERA PATIO TERRAZA FLORES
251283,kKN0zE6H/OvQngzg9evWBQ==,2021-08-23,2021-09-15,2021-08-23,NaN,NaN,Capital Federal,Villa Urquiza,NaN,NaN,NaN,Venta,Departamento,2.00,1.00,46.00,43.00,USD,Departamento - Villa Urquiza
360251,9OfQxzDYdETLlAbCyDoB8A==,2021-06-11,2021-06-18,2021-06-11,-34.59,-58.41,Capital Federal,Recoleta,NaN,NaN,NaN,Venta,Departamento,3.00,2.00,39.00,38.00,USD,VENTA TRES AMBIENTES RECOLETA


### Tipos de variables

Se analiza el tipo de variable de cada columna del dataset:

In [ ]:
prop_caba_data.columns.to_list()

['id',
 'start_date',
 'end_date',
 'created_on',
 'latitud',
 'longitud',
 'place_l2',
 'place_l3',
 'place_l4',
 'place_l5',
 'place_l6',
 'operation',
 'property_type',
 'property_rooms',
 'property_bedrooms',
 'property_surface_total',
 'property_surface_covered',
 'property_currency',
 'property_title']

Se observan los siguientes tipos de variables:

**• 'id':** Cualitativa

**• 'start_date', 'created_on', 'end_date':** Cuantitativa 

**• 'latitud', 'longitud'**: Cuantitativas Continuas

**• 'place_li':** Cualitativa Nominal

**• 'operation':** Cualitativa Nominal

**• 'property_type':** Cualitativa Nominal

**• 'property_rooms', 'property_bedrooms':** Cuantitativa Discreta

**• 'property_surface_total', 'property_surface_covered', 'property_price':** Cuantitativa Continua

**• 'property_currency', 'property_title':** Cualitativa

In [ ]:
direcciones_a_borrar = ['place_l5', 'place_l6','place_l2']
cuantitativas = ['latitud', 'longitud', 'property_rooms', 'property_bedrooms', 'property_surface_total', 'property_surface_covered']
campos_direccion = set(['place_l2', 'place_l3', 'place_l4', 'place_l5','place_l6']) - set(direcciones_a_borrar)
categoricas = ['property_type'] + list(campos_direccion)
texto = ['property_title']
fecha = ['start_date', 'end_date', 'created_on']

Las variables categóricas son los campos de dirección (Provincia, Ciudad, Barrio, Zona, etc...), los tipos de propiedad, el tipo de operación y la moneda en la que está publicado el precio de venta de la propiedad.

Las variables numéricas son latitud y longitud, property_surface_total, property_surface_covered, property_rooms, property_bedrooms y property_price. A su vez las fechas también lo son, se consideran variables cuantitativas discretas (permiten realizar por ejemplo cálculos de intervalos), para las cuales podría resultar de interés calcular la duración de la publicación.

### Variables irrelevantes

En primer lugar, las columnas que se utilizaron para filtrar el DataSet original tendrán un único valor posible que no aportará información útil. Por lo detallado anteriormente, se descartan **place_l2, property_currency, operation**, ya que éstas toman los valores únicos: 'Capital Federal' , 'USD', 'Venta' respectivamente.

Luego, en el resumen conciso del DataFrame obtenido se puede observar que tanto **place_l5** como **place_l6** tienen el 100% de sus datos nulos, por lo que también se descartan.

In [ ]:
assert(len(prop_caba_data[prop_caba_data.id.isna()])==0)
def unused_cols(prop_caba_data):
  prop_caba_data = prop_caba_data.drop('operation', axis=1)
  prop_caba_data = prop_caba_data.drop('property_currency', axis=1)
  prop_caba_data = prop_caba_data.drop('place_l2', axis=1)
  prop_caba_data = prop_caba_data.drop('place_l5', axis=1)
  prop_caba_data = prop_caba_data.drop('place_l6', axis=1)
  return prop_caba_data

In [ ]:
prop_caba_data = unused_cols(prop_caba_data)
prop_caba_test = unused_cols(prop_caba_test)

Por otro lado, se considera que **id** es también irrelevante, ya que es un hash único que no parece ser usado en el resto del dataset. Si bien el hash es el mismo que tienen las propiedades en el dataset que contiene las descripciones, se toma la decisión de utilizar el index de las filas y no el id.

In [ ]:
prop_caba_data = prop_caba_data.drop('id', axis=1)
prop_caba_test = prop_caba_test.drop('id', axis=1)

Se considera que la variable 'created_on' es irrelevante ya que las ocurrencias coinciden en su totalidad con el valor 'start_date'

In [ ]:
len(prop_caba_data.loc[prop_caba_data.start_date !=prop_caba_data.created_on])

0

De todos modos, para la realización del presente trabajo práctico se ignoran las fechas de inicio y fin de publicación de una propiedad, considerando que no tienen efecto en el precio de las mismas.

In [ ]:
for col in fecha:
  prop_caba_data = prop_caba_data.drop(col, axis=1)
  prop_caba_test = prop_caba_test.drop(col, axis=1)

A su vez, **property_title** se toma como irrelevante ya que no es posible extraer conclusiones directas a partir del texto que representa.

In [ ]:
prop_caba_data = prop_caba_data.drop('property_title', axis=1)
prop_caba_test = prop_caba_test.drop('property_title', axis=1)

### Variables cuantitativas
Describe() nos da la media, mediana, cuartiles...

In [ ]:
prop_caba_data[cuantitativas].describe()

,latitud,longitud,property_rooms,property_bedrooms,property_surface_total,property_surface_covered
count,"72,553.00","72,553.00","74,687.00","66,792.00","71,627.00","72,840.00"
mean,-34.59,-58.45,2.82,2.01,109.10,88.38
std,0.18,0.29,1.43,1.13,947.04,"1,007.55"
min,-53.79,-71.47,1.00,-1.00,10.00,1.00
25%,-34.62,-58.47,2.00,1.00,45.00,40.00
50%,-34.60,-58.44,3.00,2.00,65.00,58.00
75%,-34.58,-58.41,4.00,3.00,105.00,88.00
max,-24.22,-54.13,40.00,70.00,"137,000.00","217,000.00"


Ahora para calcular Q1, Medianas y Q3:

In [ ]:
prop_caba_data[cuantitativas].quantile([0.25, 0.5, 0.75])

,latitud,longitud,property_rooms,property_bedrooms,property_surface_total,property_surface_covered
0.25,-34.62,-58.47,2.00,1.00,45.00,40.00
0.50,-34.60,-58.44,3.00,2.00,65.00,58.00
0.75,-34.58,-58.41,4.00,3.00,105.00,88.00


Para el calculo de la moda se utiliza el método:

In [ ]:
prop_caba_data[cuantitativas].mode()

,latitud,longitud,property_rooms,property_bedrooms,property_surface_total,property_surface_covered
0,-34.54,-58.47,3.00,1.00,50.00,40.00


A continuación se realizan los gráficos de distribuciones de variables cuantitativas

In [ ]:
lat_den = sns.histplot(prop_caba_data['latitud'], bins= 2500).set(title="Distribución de la variable longitud",
                                                            xlabel="latitud",
                                                            ylabel="Densidad",
                                                            xlim=(-34.71, -34.5))

ImportError: ignored

<Figure size 432x288 with 1 Axes>

In [ ]:
long_den = sns.histplot(prop_caba_data['longitud'], bins = 1500).set(title="Distribución de la variable longitud",
                                                            xlabel="longitud",
                                                            ylabel="Densidad",
                                                            xlim=(-58.55, -58.32))

ImportError: ignored

<Figure size 432x288 with 1 Axes>

In [ ]:
rooms_hist = sns.histplot(prop_caba_data['property_rooms'], discrete=True).set(title="Distribución de la variable property_rooms",
                                                            xlabel="Cantidad de Ambientes",
                                                            ylabel="Frecuencia",
                                                            xlim=(0, 10))

ImportError: ignored

<Figure size 432x288 with 1 Axes>

In [ ]:
bedrooms_hist = sns.histplot(prop_caba_data['property_bedrooms'], discrete=True).set(title="Distribución de la variable property_bedrooms",
                                                            xlabel="Cantidad de Habitaciones",
                                                            ylabel="Frecuencia",
                                                            xlim=(0, 10))

ImportError: ignored

<Figure size 432x288 with 1 Axes>

In [ ]:
sup_total_den = sns.histplot(prop_caba_data, x='property_surface_total', log_scale=True, bins = 50).set(title="Distribución de la variable property_surface_total",
                                                            xlabel="Superficie Total",
                                                            ylabel="Frecuencia",
                                                            xlim=(10, 2000))

ImportError: ignored

<Figure size 432x288 with 1 Axes>

In [ ]:
sup_cubierta_den = sns.histplot(prop_caba_data, x='property_surface_covered', log_scale=True, bins=50).set(title="Distribución de la variable property_surface_covered",
                                                            xlabel="Superficie Cubierta",
                                                            ylabel="Frecuencia",
                                                            xlim=(10, 2000))

ImportError: ignored

<Figure size 432x288 with 1 Axes>

In [ ]:
price_den = sns.histplot(data=train_target, log_scale=True, bins=50).set(title="Distribución de la variable property_price",
                                                    xlabel="Precio",
                                                    ylabel="Frecuencia",
                                                    xlim=(20000, 5000000))
#plt.ticklabel_format(useOffset=False, style='plain')

ImportError: ignored

<Figure size 432x288 with 1 Axes>

Se puede apreciar que el precio de la mayoría de las propiedades ronda los cien mil dolares.

In [ ]:
train_target.mode()

0             110,000.00
dtype: float64

### Variables cualitativas
Vemos los valores únicos para las variables categóricas

In [ ]:
prop_caba_data[categoricas].nunique()

property_type     3
place_l3         57
place_l4          4
dtype: int64

In [ ]:
#Tipo de Propiedad
prop_type_data = prop_caba_data['property_type'].value_counts()


sns_bar_plot=sns.barplot(x=prop_type_data.index.tolist(), y=prop_type_data.tolist() ,alpha=0.5).set(title="Cantidad de observaciones por tipo de propiedad",
                                                                      xlabel='Tipo de propiedad',
                                                                      ylabel='Frecuencia')

ImportError: ignored

<Figure size 432x288 with 1 Axes>

In [ ]:
# Barrio
barrio_data = prop_caba_data['place_l3'].value_counts()
barrios = barrio_data.index.tolist()

plt.figure(figsize = (10,10))
sns_bar_plot=sns.barplot(x=barrio_data.tolist(), y=barrios , orient="h").set(title="Frecuencia de observaciones por barrio",
                                                                      xlabel='Observaciones',
                                                                      ylabel='Barrio')

ImportError: ignored

<Figure size 720x720 with 1 Axes>

place_l4 sólo aplica a Palermo y lo subdivide en cuatro sub-zonas.

In [ ]:
prop_caba_data['place_l4'].value_counts(dropna=False)

NaN                  72600
Palermo Hollywood     1274
Palermo Chico          878
Palermo Soho           633
Palermo Viejo          132
Name: place_l4, dtype: int64

In [ ]:
# Sub Barrio
sub_barrio_data = prop_caba_data['place_l4'].value_counts()
sub_barrios = sub_barrio_data.index.tolist()

sns_bar_plot=sns.barplot(x=sub_barrio_data.tolist(), y=sub_barrios , orient="h").set(title="Frecuencia de observaciones por sub barrio de Palermo",
                                                                      xlabel='Observaciones',
                                                                      ylabel='Sub Barrio')

ImportError: ignored

<Figure size 432x288 with 1 Axes>

In [ ]:
prop_caba_data[prop_caba_data.place_l4.notna()].place_l3.value_counts()

Palermo    2917
Name: place_l3, dtype: int64

In [ ]:
prop_caba_data[prop_caba_data.place_l4.notna()].place_l3.isna().any()

False

Por lo tanto place_l4 sólo aplica a propiedades en Palermo. Pero de todos modos, no se considera irrelevante ya que se toma la decisión de sustituír las ocurrencias de 'Palermo' en place_l3 por las ocurrencias de place_l4 donde ésta tiene un valor no nulo.

### Correlaciones

A continuación se realiza un análisis de correlaciones entre las variables del Dataset de propiedades filtrado. Para ello se utiliza un mapa de calor de correlación, este tipo de gráfico visualiza la fuerza de las relaciones entre variables numéricas, y se utiliza para comprender qué variables están relacionadas entre sí y la fuerza de esta relación.

In [ ]:
if not quick_exec:
  plt.figure(figsize = (10,8))
  corr = prop_caba_data.corr()
  sns.heatmap(corr, annot=True)


ImportError: ignored

<Figure size 720x576 with 2 Axes>

Se puede apreciar en el gráfico que las variables más correlacionadas son 'property_rooms' con 'property_bedrooms', esta correlación es prácticamente lineal (0.87).

In [ ]:
table = prop_caba_data.pivot_table(index='property_rooms', columns='property_bedrooms',values='latitud', aggfunc='count', fill_value=0)
table_log=table.applymap(lambda x: math.log10(x) if x > 0 else 0)

In [ ]:
if not quick_exec:
  sns.heatmap(table_log).set(title="Properties by number of bedrooms/rooms")

ImportError: ignored

<Figure size 432x288 with 2 Axes>

Los datos de beds y bedrooms están muy sesgados a la izquierda.

En segundo lugar, 'property_surface_total' y 'property_surface_covered' también están considerablemente correlacionadas (0.51)

In [ ]:
def areamap(x):
  return round(x, -1)
if not quick_exec:
  binned_surf = prop_caba_data[['property_surface_covered','property_surface_total']].applymap(areamap)
  binned_surf = binned_surf.assign(values=1)
  table_surf =  binned_surf.pivot_table(index='property_surface_total', columns='property_surface_covered', values='values', aggfunc='count',fill_value=0)
  table_log=table_surf.applymap(lambda x: math.log10(x) if x > 0 else 0)
  table_log.head()

In [ ]:
if not quick_exec:
  sns.heatmap(table_log)
  plt.ylim(50,0)
  plt.xlim(0,50)
  plt.show()

ImportError: ignored

<Figure size 432x288 with 2 Axes>

In [ ]:
if not quick_exec:
  sns.heatmap(table_log)
  plt.ylim(10,0)
  plt.xlim(0,10)
  plt.show()

ImportError: ignored

<Figure size 432x288 with 2 Axes>

##**b) Datos Faltantes :**

Veo el porcentaje de valores nulos de cada atributo

In [ ]:
prop_caba_data.shape

(75517, 9)

In [ ]:
def percentage_null_per_column(df):
  return df.isna().agg('sum', axis=0) * 100 / len(df)

In [ ]:
percentage_null_per_column(prop_caba_data)

latitud                                    3.92
longitud                                   3.92
place_l3                                   0.44
place_l4                                  96.14
property_type                              0.00
property_rooms                             1.10
property_bedrooms                         11.55
property_surface_total                     5.15
property_surface_covered                   3.54
dtype: float64

Veamos los datos faltantes según el tipo de propiedad.

In [ ]:
grouped_data = prop_caba_data.groupby('property_type')
for group in grouped_data.groups:
  print(group)
  print(percentage_null_per_column(grouped_data.get_group(group)))
  print('\n')

Casa
latitud                                    2.98
longitud                                   2.98
place_l3                                   4.11
place_l4                                  98.18
property_type                              0.00
property_rooms                             8.73
property_bedrooms                          3.58
property_surface_total                    10.31
property_surface_covered                   2.71
dtype: float64


Departamento
latitud                                    4.26
longitud                                   4.26
place_l3                                   0.27
place_l4                                  95.75
property_type                              0.00
property_rooms                             0.75
property_bedrooms                         12.95
property_surface_total                     5.01
property_surface_covered                   3.71
dtype: float64


PH
latitud                                    1.43
longitud                         

### Missing values: place_l4
place_l4 tiene muchos valores perdidos porque sólo descibe subregiones de Palermo.

In [ ]:
prop_caba_data.place_l3.value_counts()[:5]

Palermo          11444
Belgrano          7067
Caballito         5770
Recoleta          5088
Villa Urquiza     3742
Name: place_l3, dtype: int64

In [ ]:
prop_caba_data.place_l4.value_counts()[:5]

Palermo Hollywood    1274
Palermo Chico         878
Palermo Soho          633
Palermo Viejo         132
Name: place_l4, dtype: int64

### Valores nulos de ambientes y habitaciones
Las columnas de cantidad de 'ambientes' (rooms) y  'habitaciones' (bedrooms) tienen muchos valores nulos. Vemos los registros donde faltan ambas:


In [ ]:
room_data_missing = prop_caba_data[prop_caba_data.property_rooms.isna() & prop_caba_data.property_bedrooms.isna()]

In [ ]:
len(room_data_missing)

340

 Vemos una donde falta alguna, o rooms o bedrooms

In [ ]:
len(prop_caba_data[prop_caba_data.property_rooms.isna() | prop_caba_data.property_bedrooms.isna()])

9215

### Valores faltantes por fila
A cuántas filas les falta un dado porcentaje de sus atributos?

In [ ]:
def quantity_null_per_row(df):
  return df.isna().agg('sum', axis=1)

In [ ]:
percentage_missing = (quantity_null_per_row(prop_caba_data) * 100 / len(csv.columns))

In [ ]:
percentage_missing.head()

100248                   5.00
251283                  15.00
360251                   5.00
97804                   10.00
1472                     5.00
dtype: float64

In [ ]:
missing_values = percentage_missing.value_counts().sort_index()

In [ ]:
missing_values.index = missing_values.index.map(lambda label: str(round(label)) + '%')

In [ ]:
#cantidad de columnas
len(prop_caba_data.columns)

9

In [ ]:
sns.barplot(x=missing_values.index, y = missing_values.values)
plt.title("Rows by % of missing values")
plt.ylabel("number of rows")
plt.xlabel("% missed values")

Text(0.5, 0, '% missed values')

ImportError: ignored

<Figure size 432x288 with 1 Axes>

En este gráfico se ve que a la mayoría de los registros le faltan 5% de los atributos

### Otros valores faltantes


In [ ]:
prop_caba_data.property_bedrooms.unique()

array([ 5.,  1.,  2.,  3.,  4., nan,  6., 17., 10.,  7.,  8., 20., 30.,
        9., 22., 13., 11., 70., 15., 14., 18., 16., 12., -1., 32.])

In [ ]:
prop_caba_data.property_rooms.unique()

array([ 7.,  2.,  3.,  4.,  5.,  1.,  6., nan,  8., 12., 10., 18.,  9.,
       11., 14., 15., 17., 36., 20., 19., 13., 30., 35., 16., 40., 32.,
       24.])

Se ve que bedrooms tiene el valor negativo -1, lo consideramos faltante.

### Estrategias para reparar valores faltantes

### rooms y bedrooms

#### Faltan ambas

In [ ]:
room_cols = ['property_rooms', 'property_bedrooms']

In [ ]:
median_rooms_res = prop_caba_data[room_cols].median()

In [ ]:
median_rooms_res

property_rooms                      3.00
property_bedrooms                   2.00
dtype: float64

Las medianas son razonables. Si faltan ambos valores, rellenamos con ellas:

In [ ]:
old_csv = prop_caba_data.copy(deep=True)

In [ ]:
# imputamos la columna usando fillna
new = prop_caba_data[prop_caba_data.property_rooms.isna() & prop_caba_data.property_bedrooms.isna()].property_bedrooms.fillna(median_rooms_res.property_bedrooms)
# sobreescribimos el dataset con la columna post-imputar
prop_caba_data.update(new)

In [ ]:
new_test = prop_caba_test[prop_caba_test.property_rooms.isna() & prop_caba_test.property_bedrooms.isna()].property_bedrooms.fillna(median_rooms_res.property_bedrooms)
prop_caba_test.update(new_test)

#### Hot Deck Imputing
Usamos la relación entre rooms y bedrooms para predecir una con la otra.

Definimos grupos de 'similaridad' como aquellos donde el valor de 'bedroom' es el mismo. Armamos una tabla de la mediana de rooms, para cada grupo, lo usamos para rellenar registros que les falta el valor de 'room'.

Si no hay ningun grupo con el mismo valor, usamos el grupo mas parecido.

Mismo procedimiento para registros que les falta el valor 'bedroom'.

In [ ]:
def closest_value(list, value_chosen):
  return min(list, key=lambda x:abs(x-value_chosen))
def hdi(prop_caba_data, criterion_col, target_col):
  similarity_criterion = prop_caba_data[criterion_col]
  modes = prop_caba_data.groupby(similarity_criterion)[target_col].agg(np.median)
  lacking_rows = prop_caba_data[prop_caba_data[target_col].isna()]
  newvalues = lacking_rows[criterion_col].map(lambda x: modes[x] if modes[x] else modes[closest_value(modes.keys, x)])
  return newvalues

In [ ]:
newrooms = hdi(prop_caba_data, 'property_bedrooms', 'property_rooms')
newbedrooms = hdi(prop_caba_data, 'property_rooms', 'property_bedrooms')

prop_caba_data.property_rooms.update(newrooms)
prop_caba_data.property_bedrooms.update(newbedrooms)

In [ ]:
len(prop_caba_data[prop_caba_data.property_rooms.isna()])

0

In [ ]:
len(prop_caba_data[prop_caba_data.property_bedrooms.isna()])

1

Repetir todo para test

In [ ]:
newrooms = hdi(prop_caba_test, 'property_bedrooms', 'property_rooms')
newbedrooms = hdi(prop_caba_test, 'property_rooms', 'property_bedrooms')

prop_caba_test.property_rooms.update(newrooms)
prop_caba_test.property_bedrooms.update(newbedrooms)

In [ ]:
len(prop_caba_test[prop_caba_test.property_rooms.isna()])

1

In [ ]:
len(prop_caba_test[prop_caba_test.property_bedrooms.isna()])

0

Rellenamos los registros que quedan con fillna

In [ ]:
new = prop_caba_data.property_bedrooms.fillna(median_rooms_res.property_bedrooms)
prop_caba_data.update(new)
new = prop_caba_data.property_rooms.fillna(median_rooms_res.property_rooms)
prop_caba_data.update(new)

In [ ]:
new_test = prop_caba_test.property_bedrooms.fillna(median_rooms_res.property_bedrooms)
prop_caba_test.update(new_test)
new_test = prop_caba_test.property_rooms.fillna(median_rooms_res.property_rooms)
prop_caba_test.update(new_test)

### Comparamos las distribuciones antes y despues

In [ ]:
old_csv[room_cols].describe()

,property_rooms,property_bedrooms
count,"74,687.00","66,792.00"
mean,2.82,2.01
std,1.43,1.13
min,1.00,-1.00
25%,2.00,1.00
50%,3.00,2.00
75%,4.00,3.00
max,40.00,70.00


In [ ]:
prop_caba_data[room_cols].describe()

,property_rooms,property_bedrooms
count,"75,517.00","75,517.00"
mean,2.83,1.91
std,1.44,1.11
min,1.00,-1.00
25%,2.00,1.00
50%,3.00,2.00
75%,4.00,3.00
max,40.00,70.00


In [ ]:
old_csv[room_cols].hist(bins=range(0,30), figsize=(20,5))

plt.ylabel('Density')

Text(0, 0.5, 'Density')

ImportError: ignored

<Figure size 1440x360 with 2 Axes>

In [ ]:
prop_caba_data[room_cols].hist(bins=range(0,30), figsize=(20,5))

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x7fc9ad105a60>,
      dtype=object)

ImportError: ignored

<Figure size 1440x360 with 2 Axes>

### place_l4
place_l3 son barrios de la ciudad y l4 son sub-barrios dentro de palermo. Podemos unirlos en un solo atributo, con los atributos de l3, y para las propiedades en palermo tiene la subregión de palermo en la que están

In [ ]:
indexer = prop_caba_data.place_l4.notna()
indexer_test = prop_caba_test.place_l4.notna()

prop_caba_data.loc[indexer,:]

,latitud,longitud,place_l3,place_l4,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered
228951,-34.58,-58.44,Palermo,Palermo Hollywood,Departamento,2.00,1.00,52.00,52.00
196297,-34.58,-58.41,Palermo,Palermo Chico,Departamento,6.00,4.00,212.00,192.00
40673,-34.58,-58.42,Palermo,Palermo Chico,Departamento,3.00,2.00,86.00,79.00
293147,-34.58,-58.43,Palermo,Palermo Hollywood,Departamento,1.00,1.00,42.00,34.00
279823,-34.58,-58.44,Palermo,Palermo Hollywood,Departamento,2.00,1.00,NaN,52.00
...,...,...,...,...,...,...,...,...,...
55039,-34.58,-58.44,Palermo,Palermo Hollywood,Departamento,1.00,1.00,29.00,29.00
409237,-34.59,-58.42,Palermo,Palermo Soho,Departamento,1.00,1.00,NaN,48.00
316300,-34.58,-58.41,Palermo,Palermo Chico,Departamento,1.00,1.00,22.00,22.00
110783,-34.58,-58.42,Palermo,Palermo Chico,Departamento,4.00,3.00,80.00,70.00


In [ ]:
# Para todas las propiedades donde l4 tiene un valor, sustituir ese valor en l3 
prop_caba_data.loc[indexer, "place_l3"] = prop_caba_data.loc[indexer,"place_l4"]
#test
prop_caba_test.loc[indexer_test, "place_l3"] = prop_caba_test.loc[indexer_test,"place_l4"]

In [ ]:
# Veamos todos los lugares donde se aplicó la transformación
prop_caba_data[prop_caba_data.place_l4.notna()].place_l3.value_counts(dropna= False)

Palermo Hollywood    1274
Palermo Chico         878
Palermo Soho          633
Palermo Viejo         132
Name: place_l3, dtype: int64

In [ ]:
prop_caba_data = prop_caba_data.drop('place_l4',axis='columns')
prop_caba_test = prop_caba_test.drop('place_l4',axis='columns')

### sin place_l3
rellenamos con moda de place_l3

In [ ]:
mode_place =  prop_caba_data.place_l3.mode()

In [ ]:
prop_caba_data.loc[prop_caba_data.place_l3.isna(),'place_l3'] = mode_place.iloc[0]
#test
prop_caba_test.loc[prop_caba_test.place_l3.isna(),'place_l3'] = mode_place.iloc[0]

### Latitud y longitud
Cuando falta o longitud o longitud, faltan ambos.
Alrededor de un 10% de datos tiene latitud y longitud faltante. Eliminamos los faltantes
 TODO: rellenar con mediana por cada región l3

In [ ]:
old_csv = prop_caba_data.copy()

In [ ]:
(prop_caba_data[prop_caba_data.latitud.isna()].index == prop_caba_data[prop_caba_data.longitud.isna()].index).all()

True

In [ ]:
len(prop_caba_data[prop_caba_data.latitud.isna()])

2964

Agrupar por región l3, obtener tabla de medianas

In [ ]:
prop_caba_data.groupby('place_l3')[['longitud', 'latitud']].median()

,longitud,latitud
place_l3,,
Abasto,-58.41,-34.60
Agronomía,-58.49,-34.59
Almagro,-58.42,-34.61
Balvanera,-58.40,-34.61
Barracas,-58.37,-34.64
...,...,...
Villa Riachuelo,-58.47,-34.69
Villa Santa Rita,-58.48,-34.62
Villa Soldati,-58.44,-34.66


Usar tabla para rellenar valores

In [ ]:
def _get_table(X, col1, col2):
  return X.groupby(X[col1])[col2].agg(np.median)
def hdi(prop_caba_data, table, col1, col2):
  lacking_rows = prop_caba_data[prop_caba_data[col2].isna()]
  newvalues = lacking_rows[col1].map(lambda x: table[x] if table[x] else prop_caba_data[col2].median())
  return newvalues

In [ ]:
table = _get_table(prop_caba_data, 'place_l3', 'longitud')
newlon = hdi(prop_caba_data, table, 'place_l3', 'longitud')
newlat = hdi(prop_caba_data, table, 'place_l3', 'latitud')

prop_caba_data.longitud.update(newlon)
prop_caba_data.latitud.update(newlat)

In [ ]:
# test, usando la misma tabla
newlon = hdi(prop_caba_test, table, 'place_l3', 'longitud')
newlat = hdi(prop_caba_test, table, 'place_l3', 'latitud')

prop_caba_test.longitud.update(newlon)
prop_caba_test.latitud.update(newlat)

In [ ]:
percentage_null_per_column(prop_caba_test)

latitud                                    0.00
longitud                                   0.00
place_l3                                   0.00
property_type                              0.00
property_rooms                             0.00
property_bedrooms                          0.00
property_surface_total                     5.09
property_surface_covered                   3.61
dtype: float64

repito para test:

In [ ]:
newlon = hdi(prop_caba_data, table, 'place_l3', 'longitud')
newlat = hdi(prop_caba_data, table, 'place_l3', 'latitud')

prop_caba_data.longitud.update(newlon)
prop_caba_data.latitud.update(newlat)

Comparamos distribuciones (TODO histograma)

In [ ]:
old_csv.longitud.describe()

count              72,553.00
mean                  -58.45
std                     0.29
min                   -71.47
25%                   -58.47
50%                   -58.44
75%                   -58.41
max                   -54.13
Name: longitud, dtype: float64

In [ ]:
prop_caba_data.longitud.describe()

count              75,517.00
mean                  -58.45
std                     0.28
min                   -71.47
25%                   -58.47
50%                   -58.44
75%                   -58.41
max                   -54.13
Name: longitud, dtype: float64

### Área superficial
Creamos un modelo de regresión para la superficie cubierta en base a long, lat, rooms y bedrooms

In [ ]:
len(prop_caba_data[prop_caba_data.property_surface_total.isna()])

3890

In [ ]:
len(prop_caba_data[prop_caba_data.property_surface_covered.isna()])

2677

In [ ]:
from sklearn import preprocessing
def join_xy_nonull(X,y):
  joined = X.join(y, how='inner')
  joined = joined.dropna()
  X = joined[X.columns]
  y = joined[y.name]
  return X,y

def train_scaler(X):
  scaler = preprocessing.MinMaxScaler()
  scaler.fit(X.values)
  return scaler

def regress_column(y, X):
  from sklearn import linear_model
  # train with non null dataset
  X,y = join_xy_nonull(X,y)

  #scale data
  scaler = train_scaler(X)
  X_scale = scaler.transform(X.values)
  model = linear_model.LinearRegression()
  model.fit(X=X_scale, y=y)
  return scaler, model

def model_values(model, y, input_frame):
  # missing y, no null X
  X = input_frame.dropna()
  X_no_y = X.loc[y.isna(), :]
  # predict          
  raw_predictions = model.predict(X_no_y)
  predictions = pd.Series(raw_predictions, index = X_no_y.index, name = y.name)
  return predictions

'''must be non null X and y'''
def test_model(model, X, y):
  return model.score(X=X, y = y)

In [ ]:
X_input = prop_caba_test[['property_rooms', 'property_bedrooms','longitud','latitud']]
ycovered = prop_caba_test['property_surface_covered']
ytotal = prop_caba_test['property_surface_total']

X_input_test = prop_caba_data[['property_rooms', 'property_bedrooms','longitud','latitud']]
ycovered_test = prop_caba_data['property_surface_covered']
ytotal_test = prop_caba_data['property_surface_total']

In [ ]:
# covered
X, X_eval, y, y_eval = train_test_split(X_input, ycovered, test_size=0.2,random_state=12)
median = y.median()

#train
scaler, model = regress_column(y, X)

#use
X_nn,y_nn = join_xy_nonull(X_eval,y_eval)
X_nn = scaler.transform(X_nn.values)
predictions = model.predict(X_nn)
scatter_predicted_versus_real(predictions, y_nn.values,size=4)
r2_score = test_model(model, X_nn, y_nn)

ImportError: ignored

<Figure size 432x288 with 1 Axes>

In [ ]:
r2_score

0.5179426488152368

Dependiendo del r2 score en train, uso o no el modelo en los conjuntos train y test

In [ ]:
values = model_values(model, y, X)
if(r2_score > 0.4):
  prop_caba_data.update(values)
ycovered.fillna(median, inplace=True)

#aplicar modelo a test
X = X_input_test
y = ycovered_test
values = model_values(model, y, X)

# si el r2 score en train dio bien, uso el modelo
if(r2_score > 0.4):
  prop_caba_test.update(values)
ycovered_test.fillna(median, inplace=True)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


Repetir para superficie total

In [ ]:
# total
X, X_eval, y, y_eval = train_test_split(X_input, ytotal, test_size=0.2,random_state=12)
median = y.median()

#train
scaler, model = regress_column(y, X)

#use
X_nn,y_nn = join_xy_nonull(X_eval,y_eval)
X_nn = scaler.transform(X_nn.values)
predictions = model.predict(X_nn)
scatter_predicted_versus_real(predictions, y_nn.values,size=4)
r2_score = test_model(model, X_nn, y_nn)

ImportError: ignored

<Figure size 432x288 with 1 Axes>

In [ ]:
values = model_values(model, y, X)
if(r2_score > 0.4):
  prop_caba_data.update(values)
ytotal.fillna(median, inplace=True)

#aplicar modelo a test
X = X_input_test
y = ytotal_test
values = model_values(model, y, X)
if(r2_score > 0.4):
  prop_caba_test.update(values)
ytotal_test.fillna(median, inplace=True)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


Vemos proporción de nulos después del preprocesamiento:

In [ ]:
percentage_null_per_column(prop_caba_data)

latitud                                    0.00
longitud                                   0.00
place_l3                                   0.00
property_type                              0.00
property_rooms                             0.00
property_bedrooms                          0.00
property_surface_total                     0.00
property_surface_covered                   0.00
dtype: float64

In [ ]:
percentage_null_per_column(prop_caba_test)

latitud                                    0.00
longitud                                   0.00
place_l3                                   0.00
property_type                              0.00
property_rooms                             0.00
property_bedrooms                          0.00
property_surface_total                     0.00
property_surface_covered                   0.00
dtype: float64

#### Distribución tras transformaciones

In [ ]:
old_csv[['property_surface_total', 'property_surface_covered']].describe()

,property_surface_total,property_surface_covered
count,"71,627.00","72,840.00"
mean,109.10,88.38
std,947.04,"1,007.55"
min,10.00,1.00
25%,45.00,40.00
50%,65.00,58.00
75%,105.00,88.00
max,"137,000.00","217,000.00"


In [ ]:
prop_caba_data[['property_surface_total', 'property_surface_covered']].describe()

,property_surface_total,property_surface_covered
count,"75,517.00","75,517.00"
mean,106.82,87.27
std,922.37,989.54
min,10.00,1.00
25%,46.00,41.00
50%,65.00,57.00
75%,100.00,86.00
max,"137,000.00","217,000.00"


In [ ]:
# quitmoslos place_l3 nulos
prop_caba_data = prop_caba_data.loc[prop_caba_data.place_l3.notna(), :]
#test
prop_caba_test = prop_caba_test.loc[prop_caba_test.place_l3.notna(), :]

Finalmente tenemos estos valores nulos:

In [ ]:
percentage_null_per_column(prop_caba_data)

latitud                                    0.00
longitud                                   0.00
place_l3                                   0.00
property_type                              0.00
property_rooms                             0.00
property_bedrooms                          0.00
property_surface_total                     0.00
property_surface_covered                   0.00
dtype: float64

In [ ]:
percentage_null_per_column(prop_caba_test)

latitud                                    0.00
longitud                                   0.00
place_l3                                   0.00
property_type                              0.00
property_rooms                             0.00
property_bedrooms                          0.00
property_surface_total                     0.00
property_surface_covered                   0.00
dtype: float64

##**c) Valores atípicos :**

In [ ]:
prop_caba_data.shape

(75517, 8)

In [ ]:
prop_caba_test.shape

(18732, 8)

### Univariados
Usaremos en longitud/latitud, rooms/bedrooms y property_surface técnicas para identificar outliers univariados.

#### Propiedades fuera de CABA

Las regiones que difieren por mas de 1 de las coordenadas -34, -58 seguramente no están en CABA, las tiro. (Esto es una estrategia pero se aplica antes de hacer el análisis con z-score)

In [ ]:
def crop_caba_square(prop_caba_data):
  return prop_caba_data[(prop_caba_data.latitud < -33) &(prop_caba_data.latitud > -35) & (prop_caba_data.longitud < -57) &(prop_caba_data.longitud > -59) ]

prop_caba_data = crop_caba_square(prop_caba_data)
# test
prop_caba_test = crop_caba_square(prop_caba_test)

In [ ]:
prop_caba_data.shape

(72396, 8)

In [ ]:
prop_caba_test.shape

(17928, 8)

#### Sacar outliers de rooms, bedrooms, property surface
Rooms, Bedrooms y property surface tienen una distribución que favorece valores pequeños, mientras que los valores grandes son cada vez menos comunes. La distribución tiene un sesgo, podemos usar una transformación para lograr normalidad (ingeniería de features) y usar este nuevo feature para clasificar outliers.

Por esto, para obtener outliers, le aplicamos a cada variable un logaritmo, y después le aplicamos la transformación z-score. Los valores cuya magnitud de z-score supere un threshold, se consideran outliers.

In [ ]:
ZSCORE_TH = 2.5
import scipy.stats as stats

In [ ]:
def get_outliers(series, ZSCORE_TH=ZSCORE_TH):
  # normalize using log
  log = series.map(lambda x: math.log10(x) if x>0 else np.nan)
  log_z = stats.zscore(log, nan_policy='omit')
  min = series[(-ZSCORE_TH < log_z)][ (log_z <ZSCORE_TH)].min()
  max = series[(-ZSCORE_TH < log_z)][ (log_z <ZSCORE_TH)].max()
  return min, max

In [ ]:
range_rooms = get_outliers(prop_caba_data.property_rooms)
range_beds = get_outliers(prop_caba_data.property_bedrooms)
range_st = get_outliers(prop_caba_data.property_surface_total)
range_sc = get_outliers(prop_caba_data.property_surface_covered)

Analizamos los rangos producidos por este procedimiento:

In [ ]:
def index_outliers(row, range):
  return (row > range[1]) | (row < range[0])
def outliers_from_range(df, row, range):
  return df[(row > range[1]) | (row < range[0])]

In [ ]:
# cuántos valores de 'property_bedrooms' son outliers?
len(outliers_from_range(prop_caba_data, prop_caba_data.property_bedrooms, range_beds))

411

In [ ]:
# cuántos valores de 'property_rooms' son outliers?
len(outliers_from_range(prop_caba_data, prop_caba_data.property_rooms, range_rooms))

311

In [ ]:
print(range_beds)

(1.0, 5.0)


In [ ]:
print(range_rooms)

(1.0, 8.0)


In [ ]:
print(range_st)

(15.0, 351.0)


In [ ]:
print(range_sc)

(14.0, 284.0)


#### Outliers del target 
El precio también tiene distribución con skew como rooms/bedrooms/surface, así que se utiliza el mismo método para hallar outliers

In [ ]:
range_price = get_outliers(train_target,1)

In [ ]:
print(range_price)

(77500.0, 330734.0)


Se analiza la cantidad de propiedades que se descartarían filtrando según el rango obtenido 

In [ ]:
len(prop_caba_data[index_outliers(train_target, range_price)])

<ipython-input-380-9b266c180605>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(prop_caba_data[index_outliers(train_target, range_price)])


19740

Es una cantidad demasiado significativa como para decidir descartarla arbitrariamente, por lo que se realizará un análisis más profundo posteriormete.

### Multivariados
Este método es apropiado para longitud y latitud. Graficamos los puntos en 2 dimensiones, se ve que los puntos crean la silueta de CABA:

In [ ]:
if not quick_exec:
  #Scatter Plot
  sns.scatterplot( x=prop_caba_data.longitud, y=prop_caba_data.latitud, marker="4", hue=prop_caba_data.property_type)
  plt.title('Dispersograma')
  plt.show()
  prop_caba_data.columns

ImportError: ignored

<Figure size 432x288 with 1 Axes>

#### LOF
Usamos local outlier factor para identificar clusters de longitud y latitud, y limpiar los puntos atípicos en el espacio.

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

In [ ]:
longlat_data= prop_caba_data[['longitud','latitud']]
longlat_test = prop_caba_test[['longitud','latitud']]

In [ ]:
#Conjunto de entrenamiento
X=longlat_data.values
#Creo el clasificador
clf = LocalOutlierFactor(n_neighbors=20,contamination=0.1)

#Genero prediccion
y_pred = clf.fit_predict(X)

#Guardo los scores
scores=clf.negative_outlier_factor_

In [ ]:
lof_train = pd.Series(y_pred, prop_caba_data.index)
lof_train.value_counts()

 1    65157
-1     7239
dtype: int64

In [ ]:
if not quick_exec:
  #Scatter Plot
  sns.scatterplot( x=prop_caba_data[y_pred == 1].longitud, y=prop_caba_data[y_pred == 1].latitud, marker="4", hue=prop_caba_data[y_pred == 1].property_type)
  plt.title('Dispersograma')
  plt.show()
  prop_caba_data.columns

ImportError: ignored

<Figure size 432x288 with 1 Axes>

Analicemos los outliers:

In [ ]:
prop_caba_data[lof_train == -1].describe()

,latitud,longitud,property_rooms,property_bedrooms,property_surface_total,property_surface_covered
count,"7,239.00","7,239.00","7,239.00","7,239.00","7,239.00","7,239.00"
mean,-34.59,-58.44,2.86,1.93,99.25,83.69
std,0.03,0.04,1.44,1.08,229.20,194.89
min,-34.70,-58.53,1.00,1.00,14.00,1.00
25%,-34.61,-58.46,2.00,1.00,46.00,41.00
50%,-34.59,-58.44,3.00,2.00,65.00,57.00
75%,-34.57,-58.41,4.00,3.00,105.00,90.00
max,-34.54,-58.34,15.00,13.00,"14,744.00","13,717.00"


In [ ]:
prop_caba_data[lof_train == 1].describe()

,latitud,longitud,property_rooms,property_bedrooms,property_surface_total,property_surface_covered
count,"65,157.00","65,157.00","65,157.00","65,157.00","65,157.00","65,157.00"
mean,-34.60,-58.44,2.82,1.91,107.14,87.41
std,0.03,0.04,1.42,1.11,989.01,"1,063.01"
min,-34.69,-58.53,1.00,-1.00,10.00,1.00
25%,-34.62,-58.47,2.00,1.00,45.00,41.00
50%,-34.60,-58.44,3.00,2.00,65.00,57.00
75%,-34.58,-58.41,4.00,3.00,100.00,85.00
max,-34.54,-58.36,40.00,70.00,"137,000.00","217,000.00"


In [ ]:
prop_caba_data[lof_train == -1].sample(5)

,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered
302785,-34.58,-58.42,Palermo,Departamento,6.00,5.00,158.00,152.00
346254,-34.61,-58.38,Monserrat,Departamento,3.00,1.00,110.00,95.00
375726,-34.62,-58.43,Caballito,Departamento,3.00,2.00,56.00,56.00
255731,-34.62,-58.46,Flores,Departamento,3.00,2.00,65.00,65.00
371344,-34.58,-58.46,Colegiales,Departamento,1.00,1.00,30.00,25.00


In [ ]:
print(scores)

[-1.00760851e+00 -1.37886348e+00 -1.00309850e+00 ... -1.14800588e+00
 -9.74177960e-01 -4.29544632e+04]


##### Test

In [ ]:
# ahora usamos el mismo modelo para test
y_pred_test = clf.fit_predict(longlat_test.values)
lof_test = pd.Series(y_pred_test, prop_caba_test.index)
scores_test=clf.negative_outlier_factor_
print(scores_test)

[-1.01829219 -1.03934579 -0.98550436 ... -1.12184741 -1.03499675
 -1.02753402]


### Técnicas para los outliers

#### Atributos mayores que otros

Se descartan las propiedades con más bedrooms que rooms, y con más surface_covered que surface_total. Esto es debido a que se espera que las habitaciones sean un subconjunto de los ambientes, y que la superficie cubierta y la superficie no cubierta sumen para dar la superficie total.

In [ ]:
def more_than(prop_caba_data):
  return prop_caba_data[(prop_caba_data.property_rooms >= prop_caba_data.property_bedrooms) & (prop_caba_data.property_surface_total >= prop_caba_data.property_surface_covered)]

In [ ]:
prop_caba_data = more_than(prop_caba_data)
prop_caba_test = more_than(prop_caba_test)

#### Filtrado de outliers según rangos obtenidos mediante el análisis de logaritmo y z-score

Se descartan los outliers numéricos usando los rangos definidos anteriormente en el set de train

In [ ]:
def zscore_filtered(prop_caba_data):
  return prop_caba_data[
      ~index_outliers(prop_caba_data.property_bedrooms, range_beds)
      & ~index_outliers(prop_caba_data.property_rooms, range_rooms) 
      & ~index_outliers(prop_caba_data.property_surface_total, range_st) 
      & ~index_outliers(prop_caba_data.property_surface_covered, range_sc)]

In [ ]:
prop_caba_data = zscore_filtered(prop_caba_data)
prop_caba_test = zscore_filtered(prop_caba_test)

#### LOF

Se descartan los outliers obtenidos en el análisis LOF

In [ ]:
prop_caba_data = prop_caba_data[lof_train == 1]
prop_caba_test = prop_caba_test[lof_test == 1]

<ipython-input-396-dcd1a0360b69>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prop_caba_data = prop_caba_data[lof_train == 1]
<ipython-input-396-dcd1a0360b69>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prop_caba_test = prop_caba_test[lof_test == 1]


#### Análisis de la relación entre el precio de venta y la superficie

En este inciso se analizará el precio por metro cuadrado (pxm2) de las propiedades

In [ ]:
#Se aplica filtrado según nuevo index a los targets
train_target = train_target.filter(prop_caba_data.index)
test_target = test_target.filter(prop_caba_test.index)

In [ ]:
#Dataset con el target, para poder analizar la relacion
with_target = pd.concat([prop_caba_data, train_target], axis = 1)
with_target_test = pd.concat([prop_caba_test, test_target], axis = 1)

In [ ]:
algorithm_pmx2 = lambda row: (row.property_price / row.property_surface_total)
pmx2 = with_target.apply(algorithm_pmx2, axis=1)
pmx2_test = with_target_test.apply(algorithm_pmx2, axis=1)

In [ ]:
pmx2.describe()

count              60,814.00
mean                2,342.11
std                   944.90
min                    42.73
25%                 1,764.71
50%                 2,212.12
75%                 2,727.27
max                37,777.78
dtype: float64

In [ ]:
bp = sns.boxplot(x=pmx2).set(title="Boxplot precio por metro cuadrado de superficie",
                                                            xlabel="pxm2")

ImportError: ignored

<Figure size 432x288 with 1 Axes>

El gráfico obtenido no permite realizar un análisis con claridad, por lo que se aplica logaritmo

In [ ]:
pxm2 = pmx2.map(lambda x: math.log10(x) if x>0 else np.nan)
pxm2_test = pmx2_test.map(lambda x: math.log10(x) if x>0 else np.nan)

In [ ]:
pxm2.describe()

count              60,814.00
mean                    3.34
std                     0.16
min                     1.63
25%                     3.25
50%                     3.34
75%                     3.44
max                     4.58
dtype: float64

In [ ]:
bp = sns.boxplot(x=pxm2).set(title="Boxplot precio por metro cuadrado de superficie",
                                                            xlabel="log(pxm2)")

ImportError: ignored

<Figure size 432x288 with 1 Axes>

A continuación se realiza filtrado según la regla 1.5 IQR. Los datos que son más de 1.5 veces el valor del rango intercuartílico o, mejor dicho, que se encuentran a esa distancia del primer y tercer cuartil, se denominan valores atípicos.

In [ ]:
# finding the 1st quartile
q1 = np.quantile(pxm2, 0.25)
 
# finding the 3rd quartile
q3 = np.quantile(pxm2, 0.75)
med = np.median(pxm2)
 
# finding the iqr region
iqr = q3-q1
 
# finding upper and lower whiskers
upper_bound = q3+(1.5*iqr)
lower_bound = q1-(1.5*iqr)
print(iqr, upper_bound, lower_bound)

0.18905623622004875 3.7193129238915104 2.9630879790113154


In [ ]:
pxm2 = pxm2.loc[lambda x : x <= upper_bound].loc[lambda x : x >= lower_bound]

In [ ]:
prop_caba_data = prop_caba_data.filter(pxm2.index, axis=0)

Se replica el filtrado para test, a partir de los bounds calculados en train.

In [ ]:
pxm2_test = pxm2_test.loc[lambda x : x <= upper_bound].loc[lambda x : x >= lower_bound]

In [ ]:
prop_caba_test = prop_caba_test.filter(pxm2_test.index , axis=0)

### Se aplica finalmente filtrado según nuevo index de datos de train y test a los targets

In [ ]:
train_target = train_target.filter(prop_caba_data.index)
test_target = test_target.filter(prop_caba_test.index)

### Se guardan los conjuntos de datos obtenidos luego de realizar el trabajo de Análisis Exploratorio y Preprocesamiento de Datos:

In [ ]:
try:
  prop_caba_data.to_csv('drive/My Drive/tp1 datos grupal/Datasets/prop_caba_train.csv', index_label="index")
  prop_caba_test.to_csv('drive/My Drive/tp1 datos grupal/Datasets/prop_caba_test.csv', index_label="index")
except:
  prop_caba_data.to_csv('prop_caba_train.csv', index_label="index")
  prop_caba_test.to_csv('prop_caba_test.csv', index_label="index")

In [ ]:
prop_caba_data

,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered
100248,-34.64,-58.46,Flores,Casa,7.00,5.00,260.00,260.00
1472,-34.61,-58.37,San Telmo,PH,3.00,2.00,68.00,40.00
117479,-34.61,-58.43,Almagro,Departamento,4.00,3.00,210.00,175.00
79144,-34.64,-58.37,Barracas,Departamento,2.00,1.00,53.00,48.00
108699,-34.61,-58.40,Congreso,Departamento,4.00,3.00,124.00,104.00
...,...,...,...,...,...,...,...,...
290962,-34.61,-58.45,Villa Crespo,Departamento,1.00,1.00,32.00,29.00
105889,-34.58,-58.40,Palermo Chico,Departamento,1.00,1.00,20.00,20.00
136266,-34.61,-58.43,Caballito,Departamento,2.00,1.00,36.00,36.00
20953,-34.59,-58.39,Recoleta,Departamento,3.00,2.00,65.00,65.00


In [ ]:
prop_caba_test

,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered
352750,-34.56,-58.46,Belgrano,Departamento,3.00,2.00,65.00,65.00
94517,-34.56,-58.46,Belgrano,Departamento,3.00,2.00,57.00,49.00
65779,-34.56,-58.47,Villa Urquiza,Departamento,3.00,2.00,72.00,63.00
179714,-34.60,-58.43,Almagro,Departamento,1.00,1.00,29.00,29.00
171352,-34.65,-58.52,Mataderos,PH,3.00,2.00,86.00,66.00
...,...,...,...,...,...,...,...,...
419168,-34.59,-58.48,Parque Chas,PH,5.00,3.00,178.00,178.00
339511,-34.58,-58.43,Palermo,Departamento,2.00,1.00,38.00,35.00
187160,-34.58,-58.44,Palermo Hollywood,Departamento,3.00,1.00,111.00,111.00
30008,-34.56,-58.47,Coghlan,Departamento,2.00,1.00,53.00,47.00


In [ ]:
pre_encode = prop_caba_data.copy()

## **Análisis del dataset obtenido luego de pre-procesar los datos**

### Variables cuantitativas
Describe() nos da la media, mediana, cuartiles...

In [ ]:
prop_caba_data[cuantitativas].describe()

,latitud,longitud,property_rooms,property_bedrooms,property_surface_total,property_surface_covered
count,"58,682.00","58,682.00","58,682.00","58,682.00","58,682.00","58,682.00"
mean,-34.60,-58.44,2.69,1.80,79.04,67.02
std,0.03,0.04,1.19,0.90,51.33,41.41
min,-34.69,-58.53,1.00,1.00,15.00,14.00
25%,-34.62,-58.47,2.00,1.00,45.00,40.00
50%,-34.60,-58.44,3.00,2.00,65.00,55.00
75%,-34.58,-58.41,3.00,2.00,94.00,79.00
max,-34.54,-58.36,8.00,5.00,351.00,283.00


Ahora para calcular Q1, Medianas y Q3:

In [ ]:
prop_caba_data[cuantitativas].quantile([0.25, 0.5, 0.75])

,latitud,longitud,property_rooms,property_bedrooms,property_surface_total,property_surface_covered
0.25,-34.62,-58.47,2.00,1.00,45.00,40.00
0.50,-34.60,-58.44,3.00,2.00,65.00,55.00
0.75,-34.58,-58.41,3.00,2.00,94.00,79.00


Para el calculo de la moda se utiliza el método:

In [ ]:
prop_caba_data[cuantitativas].mode()

,latitud,longitud,property_rooms,property_bedrooms,property_surface_total,property_surface_covered
0,-34.54,-58.47,3.00,1.00,65.00,57.00


A continuación se realizan los gráficos de distribuciones de variables cuantitativas

In [ ]:
lat_den = sns.histplot(prop_caba_data['latitud'], bins= 50).set(title="Distribución de la variable longitud",
                                                            xlabel="latitud",
                                                            ylabel="Densidad",
                                                            xlim=(-34.71, -34.5))

ImportError: ignored

<Figure size 432x288 with 1 Axes>

In [ ]:
long_den = sns.histplot(prop_caba_data['longitud'], bins = 50).set(title="Distribución de la variable longitud",
                                                            xlabel="longitud",
                                                            ylabel="Densidad",
                                                            xlim=(-58.55, -58.32))

ImportError: ignored

<Figure size 432x288 with 1 Axes>

In [ ]:
rooms_hist = sns.histplot(prop_caba_data['property_rooms'], discrete=True).set(title="Distribución de la variable property_rooms",
                                                            xlabel="Cantidad de Ambientes",
                                                            ylabel="Frecuencia")

ImportError: ignored

<Figure size 432x288 with 1 Axes>

In [ ]:
bedrooms_hist = sns.histplot(prop_caba_data['property_bedrooms'], discrete=True).set(title="Distribución de la variable property_bedrooms",
                                                            xlabel="Cantidad de Habitaciones",
                                                            ylabel="Frecuencia")

ImportError: ignored

<Figure size 432x288 with 1 Axes>

In [ ]:
sup_total_den = sns.histplot(prop_caba_data, x='property_surface_total', bins = 50).set(title="Distribución de la variable property_surface_total",
                                                            xlabel="Superficie Total",
                                                            ylabel="Frecuencia")

ImportError: ignored

<Figure size 432x288 with 1 Axes>

In [ ]:
sup_cubierta_den = sns.histplot(prop_caba_data, x='property_surface_covered', bins=50).set(title="Distribución de la variable property_surface_covered",
                                                            xlabel="Superficie Cubierta",
                                                            ylabel="Frecuencia")

ImportError: ignored

<Figure size 432x288 with 1 Axes>

In [ ]:
price_den = sns.histplot(data=train_target, log_scale=True, bins=50).set(title="Distribución de la variable property_price",
                                                    xlabel="Precio",
                                                    ylabel="Frecuencia")
#plt.ticklabel_format(useOffset=False, style='plain')

ImportError: ignored

<Figure size 432x288 with 1 Axes>

Se puede apreciar que el precio de la mayoría de las propiedades ronda los cien mil dolares.

In [ ]:
train_target.mode()

0             110,000.00
dtype: float64

### Variables cualitativas
Vemos los valores únicos para las variables categóricas

In [ ]:
prop_caba_data[['property_type', 'place_l3']].nunique()

property_type     3
place_l3         61
dtype: int64

In [ ]:
#Tipo de Propiedad
prop_type_data = prop_caba_data['property_type'].value_counts()


sns_bar_plot=sns.barplot(x=prop_type_data.index.tolist(), y=prop_type_data.tolist() ,alpha=0.5).set(title="Cantidad de observaciones por tipo de propiedad",
                                                                      xlabel='Tipo de propiedad',
                                                                      ylabel='Frecuencia')

ImportError: ignored

<Figure size 432x288 with 1 Axes>

In [ ]:
# Barrio
barrio_data = prop_caba_data['place_l3'].value_counts()
barrios = barrio_data.index.tolist()

plt.figure(figsize = (10,10))
sns_bar_plot=sns.barplot(x=barrio_data.tolist(), y=barrios , orient="h").set(title="Frecuencia de observaciones por barrio",
                                                                      xlabel='Observaciones',
                                                                      ylabel='Barrio')

ImportError: ignored

<Figure size 720x720 with 1 Axes>

### Correlaciones

A continuación se realiza un análisis de correlaciones entre las variables del Dataset de propiedades filtrado. Para ello se utiliza un mapa de calor de correlación, este tipo de gráfico visualiza la fuerza de las relaciones entre variables numéricas, y se utiliza para comprender qué variables están relacionadas entre sí y la fuerza de esta relación.

In [ ]:
if not quick_exec:
  plt.figure(figsize = (10,8))
  corr = prop_caba_data.corr()
  sns.heatmap(corr, annot=True)


ImportError: ignored

<Figure size 720x576 with 2 Axes>

Se puede apreciar en el gráfico que las variables más correlacionadas son 'property_rooms' con 'property_bedrooms' y 'property_surface_total' con 'property_surface_covered', estas correlaciónes son prácticamente lineales.

##**d) Reducción de la dimensionalidad:**

En este inciso se realizará un análisis de componentes principales, es decir, un procedimiento matemático por el cual se transforma un conjunto de variables correlacionadas en un conjunto de variables no correlacionadas de menor dimensión. Estas nuevas variables se obtienen a partir de combinaciones lineales de las variables originales, y son las que se denominan Componentes Principales.

Esta técnica intenta hallar aquellas combinaciones lineales de las variables originales que maximizan la varianza (minimizan la pérdida de información original).

Tiene sentido aplicar esta técnica para el dataset de propiedades, debido a que las variables originales están correlacionadas.

El objetivo es reducir la dimensión del problema, es decir, descartar la información redundante.

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA

Se seleccionan columnas que tienen buena correlación para realizar el PCA.

Se crea un modelo a partir del set train.

In [ ]:
# elegir columnas y escalar
columnas_elegidas =  ['property_rooms', 'property_bedrooms', 'property_surface_total', 'property_surface_covered', 'latitud', 'longitud']

x = prop_caba_data.loc[:, columnas_elegidas].values
std_scale_pca = StandardScaler()
x = std_scale_pca.fit_transform(x)

# crear modelo
pca_comp_number = x.shape[1]
pca = PCA(n_components=pca_comp_number)

# fit y convertir en dataframe
x_transformation = pca.fit_transform(x)
pca_df_aux = pd.DataFrame(data = x_transformation
             , columns = ['cp_'+str(i) for i in range(0, x.shape[1])])
pca_df_aux.index = prop_caba_data.place_l3.index

Graficamos la varianza explicada de cada componente:

In [ ]:
varianza_explicada = pca.explained_variance_ratio_

# Grafico varianza explicada y cantidad de componentes principales
if not quick_exec:
  list_x = list(range(1, len(varianza_explicada)+1))
  plt.figure(figsize=(6, 4))
  plt.plot(list_x, varianza_explicada.cumsum() * 100, '-o')
  plt.xlabel(r'Número de componentes')
  plt.ylabel('Varianza Explicada (%)')
  plt.show()

ImportError: ignored

<Figure size 432x288 with 1 Axes>

### Replicar para test

In [ ]:
# elegir columnas y escalar
x = prop_caba_test.loc[:, columnas_elegidas].values
x_transformation = std_scale_pca.transform(x)

# aplicar pca entrenado
pca.transform(x)
pca_df_test = pd.DataFrame(data = x_transformation
             , columns = ['cp_'+str(i) for i in range(0, x.shape[1])])
pca_df_test.index = prop_caba_test.place_l3.index

### Gráficos

In [ ]:
#concatenar tipo para graficos
color = test_target.filter(pca_df_test.index)
df_pca = pd.concat([pca_df_test, color], axis = 1)
df_pca.columns= list(pca_df_test.columns) + [color.name]
df_pca.head()

,cp_0,cp_1,cp_2,cp_3,cp_4,cp_5,property_price
352750,0.26,0.22,-0.27,-0.05,1.49,-0.62,"150,000.00"
94517,0.26,0.22,-0.43,-0.44,1.45,-0.50,"156,000.00"
65779,0.26,0.22,-0.14,-0.10,1.40,-0.85,"193,000.00"
179714,-1.42,-0.89,-0.98,-0.92,-0.29,0.33,"68,500.00"
171352,0.26,0.22,0.14,-0.02,-2.16,-2.03,"180,000.00"


Graficamos los componentes principales para ver cómo se distribuyen

In [ ]:
if not quick_exec:
  sns.scatterplot(data=df_pca, x="cp_0", y="cp_1", hue="property_price")

ImportError: ignored

<Figure size 432x288 with 1 Axes>

In [ ]:
if not quick_exec:
  sns.scatterplot(data=df_pca, x="cp_1", y="cp_2", hue="property_price")

ImportError: ignored

<Figure size 432x288 with 1 Axes>

Analicemos la composición de cada componente principal:

In [ ]:
pca_vectors = pd.DataFrame(pca.components_, columns = columnas_elegidas)

In [ ]:
if not quick_exec:
  for i in range(0, len(columnas_elegidas)):
    plt.figure(figsize=(5,2))
    plt.barh(y=columnas_elegidas, width=pca_vectors.iloc[i])
    plt.xlim((-1,1))
    plt.title("pc "+str(i))

ImportError: ignored

<Figure size 360x144 with 1 Axes>

ImportError: ignored

<Figure size 360x144 with 1 Axes>

ImportError: ignored

...

<Figure size 360x144 with 1 Axes>

ImportError: ignored

<Figure size 360x144 with 1 Axes>

ImportError: ignored

<Figure size 360x144 with 1 Axes>

Se ve que el componente principal crece junto con las superficies y las habitaciones/ambientes. Indica que corresponden a la mayor varianza.

### Se guarda el data frame resultante de realizar la reducción de dimensión:

Se decide entonces utilizar las tres primeras componentes, debido a  que éstas explican aproximadamente el 90% de la varianza. Logrando de este modo reducir la dimensionalidad.

In [ ]:
df_PCA_train = pd.concat([pca_df_aux.loc[:, ['cp_0', 'cp_1', 'cp_2']], prop_caba_data.loc[:, ['property_type', 'place_l3']], train_target], axis=1 )
df_PCA_train

,cp_0,cp_1,cp_2,property_type,place_l3,property_price
100248,7.69,0.73,1.46,Casa,Flores,"245,000.00"
1472,-0.20,1.76,-0.73,PH,San Telmo,"90,000.00"
117479,3.79,0.58,0.02,Departamento,Almagro,"450,000.00"
79144,-1.23,2.41,-0.22,Departamento,Barracas,"125,000.00"
108699,2.10,1.23,-0.41,Departamento,Congreso,"269,000.00"
...,...,...,...,...,...,...
290962,-2.08,0.12,0.61,Departamento,Villa Crespo,"78,000.00"
105889,-2.30,0.27,-1.03,Departamento,Palermo Chico,"65,000.00"
136266,-1.53,0.49,0.20,Departamento,Caballito,"82,000.00"
20953,0.08,0.80,-0.90,Departamento,Recoleta,"140,000.00"


In [ ]:
df_PCA_test = pd.concat([pca_df_test.loc[:, ['cp_0', 'cp_1', 'cp_2']], prop_caba_test.loc[:, ['property_type', 'place_l3']], test_target], axis=1 )
df_PCA_test

,cp_0,cp_1,cp_2,property_type,place_l3,property_price
352750,0.26,0.22,-0.27,Departamento,Belgrano,"150,000.00"
94517,0.26,0.22,-0.43,Departamento,Belgrano,"156,000.00"
65779,0.26,0.22,-0.14,Departamento,Villa Urquiza,"193,000.00"
179714,-1.42,-0.89,-0.98,Departamento,Almagro,"68,500.00"
171352,0.26,0.22,0.14,PH,Mataderos,"180,000.00"
...,...,...,...,...,...,...
419168,1.95,1.34,1.93,PH,Parque Chas,"465,000.00"
339511,-0.58,-0.89,-0.80,Departamento,Palermo,"118,000.00"
187160,0.26,-0.89,0.62,Departamento,Palermo Hollywood,"395,000.00"
30008,-0.58,-0.89,-0.51,Departamento,Coghlan,"116,000.00"


In [ ]:
try:
  df_PCA_train.to_csv('drive/My Drive/tp1 datos grupal/Datasets/df_PCA_train.csv', index_label="index")
  df_PCA_test.to_csv('drive/My Drive/tp1 datos grupal/Datasets/df_PCA_test.csv', index_label="index")
except:
  df_PCA_train.to_csv('df_PCA_train.csv', index_label="index")
  df_PCA_test.to_csv('df_PCA_test.csv', index_label="index")

# Encoding

In [ ]:
prop_caba_data['place_l3'].value_counts()

Palermo            6507
Belgrano           5125
Caballito          4912
Recoleta           3934
Villa Urquiza      3073
                   ... 
Tribunales           49
Villa Riachuelo      13
Las Cañitas           7
Villa Soldati         7
Catalinas             1
Name: place_l3, Length: 61, dtype: int64

Consideramos que son demasiados barrios como para aplicar One Hot Encoding a 'place_l3' (resultaría en al rededor de 61 columnas nuevas), por lo que se decide agrupar los barrios en Comunas.

In [ ]:
comuna = {}
comuna[1] = ['Retiro', 'San Nicolás', 'Puerto Madero', 'San Telmo', 'Montserrat', 'Monserrat', 'Constitución', 'Tribunales', 'Centro / Microcentro', 'Catalinas']
comuna[2] = ['Recoleta', 'Barrio Norte']
comuna[3] = ['Balvanera', 'San Cristóbal', 'San Cristobal', 'Once', 'Congreso', 'Abasto']
comuna[4] = ['La Boca', 'Boca', 'Barracas', 'Parque Patricios', 'Nueva Pompeya', 'Pompeya']
comuna[5] = ['Almagro', 'Boedo']
comuna[6] = ['Caballito', 'Parque Centenario']
comuna[7] = ['Flores', 'Parque Chacabuco']
comuna[8] = ['Villa Soldati', 'Villa Riachuelo', 'Villa Lugano']
comuna[9] = ['Liniers', 'Mataderos', 'Parque Avellaneda']
comuna[10] = ['Villa Real', 'Monte Castro', 'Versalles', 'Floresta', 'Vélez Sarsfield', 'Velez Sarsfield', 'Villa Luro']
comuna[11] = ['Villa General Mitre', 'Villa Devoto', 'Villa del Parque', 'Villa Santa Rita']
comuna[12] = ['Coghlan', 'Saavedra', 'Villa Urquiza', 'Villa Pueyrredón']
comuna[13] = ['Núñez', 'Nuñez', 'Belgrano', 'Colegiales']
comuna[14] = ['Palermo', 'Palermo Hollywood', 'Palermo Chico', 'Palermo Soho', 'Palermo Viejo', 'Las Cañitas']
comuna[15] = ['Chacarita', 'Villa Crespo', 'La Paternal', 'Paternal', 'Villa Ortúzar', 'Villa Ortuzar', 'Agronomía', 'Parque Chas']

In [ ]:
def numero_comuna(barrio):
  for i in range(1, len(comuna.keys()) + 1):
     if barrio in comuna[i]:
        return i
  return 0

prop_caba_data['comuna'] = prop_caba_data['place_l3'].apply(numero_comuna)

In [ ]:
prop_caba_data.loc[:, ['place_l3', 'comuna']].sample(n = 5)

,place_l3,comuna
377816,Floresta,10
32118,Recoleta,2
345013,Saavedra,12
60395,Parque Chacabuco,7
54631,Recoleta,2


Se verifica que todas las propiedades fueron asignadas a una Comuna válida (comuna 0 no existe):

In [ ]:
prop_caba_data.loc[prop_caba_data.comuna == 0]

,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,comuna


Se aplica entonces One Hot Encoding a 'property_type' y a 'comuna':

In [ ]:
df_encoded = pd.get_dummies(prop_caba_data, columns=['comuna', 'property_type'], drop_first=True)

In [ ]:
df_encoded = pd.concat([df_encoded, train_target], axis=1)
df_encoded_train = df_encoded.drop(['place_l3'], axis=1)

In [ ]:
df_encoded_train

,latitud,longitud,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,comuna_2,comuna_3,comuna_4,comuna_5,...,comuna_9,comuna_10,comuna_11,comuna_12,comuna_13,comuna_14,comuna_15,property_type_Departamento,property_type_PH,property_price
100248,-34.64,-58.46,7.00,5.00,260.00,260.00,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"245,000.00"
1472,-34.61,-58.37,3.00,2.00,68.00,40.00,0,0,0,0,...,0,0,0,0,0,0,0,0,1,"90,000.00"
117479,-34.61,-58.43,4.00,3.00,210.00,175.00,0,0,0,1,...,0,0,0,0,0,0,0,1,0,"450,000.00"
79144,-34.64,-58.37,2.00,1.00,53.00,48.00,0,0,1,0,...,0,0,0,0,0,0,0,1,0,"125,000.00"
108699,-34.61,-58.40,4.00,3.00,124.00,104.00,0,1,0,0,...,0,0,0,0,0,0,0,1,0,"269,000.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290962,-34.61,-58.45,1.00,1.00,32.00,29.00,0,0,0,0,...,0,0,0,0,0,0,1,1,0,"78,000.00"
105889,-34.58,-58.40,1.00,1.00,20.00,20.00,0,0,0,0,...,0,0,0,0,0,1,0,1,0,"65,000.00"
136266,-34.61,-58.43,2.00,1.00,36.00,36.00,0,0,0,0,...,0,0,0,0,0,0,0,1,0,"82,000.00"
20953,-34.59,-58.39,3.00,2.00,65.00,65.00,1,0,0,0,...,0,0,0,0,0,0,0,1,0,"140,000.00"


Se replica el encoding para test

In [ ]:
prop_caba_test['comuna'] = prop_caba_test['place_l3'].apply(numero_comuna)

Se verifica que todas las propiedades fueron asignadas a una Comuna válida (comuna 0 no existe):

In [ ]:
prop_caba_test.loc[prop_caba_test.comuna == 0]

,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,comuna


Se aplica entinces One Hot Encoding a 'property_type' y a 'comuna':

In [ ]:
df_encoded = pd.get_dummies(prop_caba_test, columns=['comuna', 'property_type'], drop_first=True)

In [ ]:
df_encoded = pd.concat([df_encoded, test_target], axis=1)
df_encoded_test = df_encoded.drop(['place_l3'], axis=1)

In [ ]:
df_encoded_test

,latitud,longitud,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,comuna_2,comuna_3,comuna_4,comuna_5,...,comuna_9,comuna_10,comuna_11,comuna_12,comuna_13,comuna_14,comuna_15,property_type_Departamento,property_type_PH,property_price
352750,-34.56,-58.46,3.00,2.00,65.00,65.00,0,0,0,0,...,0,0,0,0,1,0,0,1,0,"150,000.00"
94517,-34.56,-58.46,3.00,2.00,57.00,49.00,0,0,0,0,...,0,0,0,0,1,0,0,1,0,"156,000.00"
65779,-34.56,-58.47,3.00,2.00,72.00,63.00,0,0,0,0,...,0,0,0,1,0,0,0,1,0,"193,000.00"
179714,-34.60,-58.43,1.00,1.00,29.00,29.00,0,0,0,1,...,0,0,0,0,0,0,0,1,0,"68,500.00"
171352,-34.65,-58.52,3.00,2.00,86.00,66.00,0,0,0,0,...,1,0,0,0,0,0,0,0,1,"180,000.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419168,-34.59,-58.48,5.00,3.00,178.00,178.00,0,0,0,0,...,0,0,0,0,0,0,1,0,1,"465,000.00"
339511,-34.58,-58.43,2.00,1.00,38.00,35.00,0,0,0,0,...,0,0,0,0,0,1,0,1,0,"118,000.00"
187160,-34.58,-58.44,3.00,1.00,111.00,111.00,0,0,0,0,...,0,0,0,0,0,1,0,1,0,"395,000.00"
30008,-34.56,-58.47,2.00,1.00,53.00,47.00,0,0,0,0,...,0,0,0,1,0,0,0,1,0,"116,000.00"


Encoding para el data set resutante de realizar la reducción de dimensionalidad

In [ ]:
df_encoded = pd.get_dummies(prop_caba_data, columns=['comuna', 'property_type'], drop_first=True)
df_PCA_ENCODED_train = pd.concat([df_PCA_train, df_encoded] ,axis = 1)
df_PCA_ENCODED_train = df_PCA_ENCODED_train.drop(['place_l3', 'property_type', 'property_rooms', 'property_bedrooms', 'property_surface_total', 'property_surface_covered', 'latitud', 'longitud'], axis=1)
df_PCA_ENCODED_train

,cp_0,cp_1,cp_2,property_price,comuna_2,comuna_3,comuna_4,comuna_5,comuna_6,comuna_7,comuna_8,comuna_9,comuna_10,comuna_11,comuna_12,comuna_13,comuna_14,comuna_15,property_type_Departamento,property_type_PH
100248,7.69,0.73,1.46,"245,000.00",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1472,-0.20,1.76,-0.73,"90,000.00",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
117479,3.79,0.58,0.02,"450,000.00",0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
79144,-1.23,2.41,-0.22,"125,000.00",0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
108699,2.10,1.23,-0.41,"269,000.00",0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290962,-2.08,0.12,0.61,"78,000.00",0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
105889,-2.30,0.27,-1.03,"65,000.00",0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
136266,-1.53,0.49,0.20,"82,000.00",0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
20953,0.08,0.80,-0.90,"140,000.00",1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
df_encoded = pd.get_dummies(prop_caba_test, columns=['comuna', 'property_type'], drop_first=True)
df_PCA_ENCODED_test = pd.concat([df_PCA_test, df_encoded] ,axis = 1)
df_PCA_ENCODED_test = df_PCA_ENCODED_test.drop(['place_l3', 'property_type', 'property_rooms', 'property_bedrooms', 'property_surface_total', 'property_surface_covered', 'latitud', 'longitud'], axis=1)
df_PCA_ENCODED_test

,cp_0,cp_1,cp_2,property_price,comuna_2,comuna_3,comuna_4,comuna_5,comuna_6,comuna_7,comuna_8,comuna_9,comuna_10,comuna_11,comuna_12,comuna_13,comuna_14,comuna_15,property_type_Departamento,property_type_PH
352750,0.26,0.22,-0.27,"150,000.00",0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
94517,0.26,0.22,-0.43,"156,000.00",0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
65779,0.26,0.22,-0.14,"193,000.00",0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
179714,-1.42,-0.89,-0.98,"68,500.00",0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
171352,0.26,0.22,0.14,"180,000.00",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419168,1.95,1.34,1.93,"465,000.00",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
339511,-0.58,-0.89,-0.80,"118,000.00",0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
187160,0.26,-0.89,0.62,"395,000.00",0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
30008,-0.58,-0.89,-0.51,"116,000.00",0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0


## Se guardan los conjuntos de datos obtenidos luego de realizar el Encoding:

In [ ]:
try:
  df_encoded_train.to_csv('drive/My Drive/tp1 datos grupal/Datasets/prop_caba_encoded_train.csv', index_label="index")
  df_encoded_test.to_csv('drive/My Drive/tp1 datos grupal/Datasets/prop_caba_encoded_test.csv', index_label="index")
except:
  df_encoded_train.to_csv('prop_caba_encoded_train.csv', index_label="index")
  df_encoded_test.to_csv('prop_caba_encoded_test.csv', index_label="index")

In [ ]:
try:
  df_PCA_ENCODED_train.to_csv('drive/My Drive/tp1 datos grupal/Datasets/df_PCA_ENCODED_train.csv', index_label="index")
  df_PCA_ENCODED_test.to_csv('drive/My Drive/tp1 datos grupal/Datasets/df_PCA_ENCODED_test.csv', index_label="index")
except:
  df_PCA_ENCODED_train.to_csv('df_PCA_ENCODED_train.csv', index_label="index")
  df_PCA_ENCODED_test.to_csv('df_PCA_ENCODED_test.csv', index_label="index")

# Modelos

Se crean Datasets con el target, los mismos se utilizan a lo largo de la realización del TP.

In [ ]:
with_target = pd.concat([prop_caba_data, train_target], axis = 1)
with_target_test = pd.concat([prop_caba_test, test_target], axis = 1)

In [ ]:
corr_train = with_target.corr()
sns.heatmap(corr_train, annot=True)

ImportError: ignored

<Figure size 432x288 with 2 Axes>

In [ ]:
corr_test = with_target_test.corr()
sns.heatmap(corr_test, annot=True)

ImportError: ignored

<Figure size 432x288 with 2 Axes>

#**2. Agrupamiento**

Seleccionamos las variables no categóricas (group cols) y solo usamos esas para el análisis no supervisado, ademas del target.

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.cluster import KMeans
from pyclustertend import hopkins

ImportError: ignored

In [ ]:
prop_caba_data.columns

In [ ]:
group_cols = ['latitud', 'longitud', 'property_rooms', 'property_bedrooms', 'property_surface_total', 'property_surface_covered'] 
pos = pd.concat([prop_caba_data[group_cols], train_target], axis = 1)
pos = pos.values
#pos = MinMaxScaler().fit_transform(pos) OBS: ESCALANDO SE OBTIENEN MALOS RESULTADOS
pos.shape

In [ ]:
pos = pd.DataFrame(pos, index=df_encoded_train.index,columns=group_cols + [train_target.name])

In [ ]:
pos.columns

## Tendencia al agrupamiento

A continuación se calcula la estadística de Hopkins, una prueba estadística que permite predecir si los datos siguen una distribución uniforme. Si la prueba es positiva (un puntaje de hopkins que tiende a 0) significa que los datos no están distribuidos uniformemente. Por lo tanto, la agrupación puede ser útil para clasificar las observaciones.

In [ ]:
hopkins(pos,10000)

El valor obtenido para la estadística de Hopkings es cercano a 0, lo que indica que los datos tienen más tendencia al clustering.

## Estimar cantidad de clusters

### Método de Elbow

El método de Elbow consiste básicamente en verificar la evolución de la suma de los cuadrados del error para varias cantidades de clusters y verificar cual es el que brinda un mejor agrupamiento

In [ ]:
# Corro el algoritmo con K entre 2 y 10 y voy guardando el error para luego graficarlo.
sse = []
list_k = list(range(1, 10))

for k in list_k:
    km = KMeans(n_clusters=k)
    km.fit(pos)
    sse.append(km.inertia_)

# Grafico el SSE por K
plt.figure(figsize=(6, 6))
plt.plot(list_k, sse, '-o')
plt.xlabel(r'Cantidad de clusters')
plt.ylabel('SSE')
plt.show()

 Si bien no se distingue un nodo en el que claramente se 'quiebre' la pendiente, se puede tomar k = 4 como cantidad de clusters debido a que para k mayores la inercia tiene una pendiente que va disminuyendo cada vez en menor medida.

### Método de Silhouette para determinar el mejor k 

In [ ]:
from yellowbrick.cluster import SilhouetteVisualizer
from sklearn.metrics import silhouette_score

In [ ]:
if quick_exec: sample = 100
else: sample = 1000
sil_dataset = pos.sample(sample, random_state=0)
best_score = 0
for n_clusters in range(2,10,1):
    clusterer = KMeans(n_clusters=n_clusters)
    clusterer.fit(sil_dataset)
    preds = clusterer.predict(sil_dataset)
    score = silhouette_score (sil_dataset, preds)
    print ("For n_clusters =" + str(n_clusters) + " silhouette score is " + str(score))
    if(score > best_score):
      best_score = score
      best_model = clusterer


In [ ]:
best_score

## Análisis de Silhouette

In [ ]:
visualizer = SilhouetteVisualizer(best_model, colors='yellowbrick')    # Instancio el visualizador

In [ ]:
visualizer.fit(sil_dataset)
plt.xlabel("Silhouette Score")
plt.ylabel("Observaciones")
plt.show()

In [ ]:
visualizer.silhouette_score_

El valor de la silueta es una medida de cuán similar es un objeto a su propio cúmulo (cohesión) en comparación con otros cúmulos (separación). 

La silueta va de -1 a +1, por lo que el valor obtenido se considera 'alto', y esto indica que los objetos están bien emparejados con su propio cúmulo y mal emparejados con los cúmulos vecinos.

## Gráficos
Grafiquemos las características para verificar la composición de los grupos.

In [ ]:
y_kmeans = best_model.predict(pos)

In [ ]:
plt.scatter(pos.iloc[:,1],pos.iloc[:,0], c=y_kmeans, s=50, cmap='viridis', marker='4')
centers = best_model.cluster_centers_
plt.scatter(centers[:,1], centers[:,0], c='black', s=200, alpha=0.7,marker='1')
plt.show()

In [ ]:
plt.scatter(pos.iloc[:,2],pos.iloc[:,3], c=y_kmeans, s=50, cmap='viridis', marker='4')
centers = best_model.cluster_centers_
plt.scatter(centers[:,2], centers[:,3], c='black', s=200, alpha=0.7,marker='1')
plt.show()

In [ ]:
if not quick_exec:
  plt.scatter(pos.iloc[:,4],pos.iloc[:,5], c=y_kmeans, s=50, cmap='viridis', marker='4')
  centers = best_model.cluster_centers_
  plt.scatter(centers[:,4], centers[:,5], c='black', s=200, alpha=0.7,marker='1')
  plt.show()

In [ ]:
sns.set_style("darkgrid")

x1 = pos['longitud']
y1 = pos['latitud']
z1 = pos['property_rooms']

plt.figure(figsize=(6,5))
axes = plt.axes(projection='3d')
print(type(axes))
axes.scatter3D(x1,y1,z1, c= y_kmeans, cmap = plt.get_cmap('Paired'))

axes.set_xlabel('lon')
axes.set_ylabel('lat')
axes.set_zlabel('rooms')
plt.show()

## Análisis de los grupos

In [ ]:
prop_caba_data[y_kmeans == 0].describe()

In [ ]:
prop_caba_data[y_kmeans == 1].describe()

In [ ]:
prop_caba_data[y_kmeans == 2].describe()

### Gráfico mapa

In [ ]:
if not quick_exec:
  pos.insert(0,'group',y_kmeans)
  pos["group"] = pos["group"].astype(str)
  fig = px.scatter_geo(data_frame=pos, lat='latitud', lon='longitud', fitbounds='locations',color='group')
  fig.show()
  pos = pos.drop('group', axis=1)

## Repetir análisis con 3 clusters

In [ ]:
best_model = KMeans(n_clusters = 3)

In [ ]:
best_model.fit(pos)

## Análisis de Silhouette

In [ ]:
visualizer = SilhouetteVisualizer(best_model, colors='yellowbrick')    # Instancio el visualizador

In [ ]:
visualizer.fit(sil_dataset)
plt.xlabel("Silhouette Score")
plt.ylabel("Observaciones")
plt.show()

In [ ]:
visualizer.silhouette_score_

## Gráficos
Grafiquemos las características para verificar la composición de los grupos.

In [ ]:
y_kmeans = best_model.predict(pos)

In [ ]:
plt.scatter(pos.iloc[:,1],pos.iloc[:,0], c=y_kmeans, s=50, cmap='viridis', marker='4')
centers = best_model.cluster_centers_
plt.scatter(centers[:,1], centers[:,0], c='black', s=200, alpha=0.7,marker='1')
plt.show()

In [ ]:
plt.scatter(pos.iloc[:,2],pos.iloc[:,3], c=y_kmeans, s=50, cmap='viridis', marker='4')
centers = best_model.cluster_centers_
plt.scatter(centers[:,2], centers[:,3], c='black', s=200, alpha=0.7,marker='1')
plt.show()

In [ ]:
plt.scatter(pos.iloc[:,4],pos.iloc[:,5], c=y_kmeans, s=50, cmap='viridis', marker='4')
centers = best_model.cluster_centers_
plt.scatter(centers[:,4], centers[:,5], c='black', s=200, alpha=0.7,marker='1')
plt.show()

In [ ]:
if not quick_exec:
  sns.set_style("darkgrid")

  x1 = pos['longitud']
  y1 = pos['latitud']
  z1 = pos['property_rooms']

  plt.figure(figsize=(6,5))
  axes = plt.axes(projection='3d')
  print(type(axes))
  axes.scatter3D(x1,y1,z1, c= y_kmeans, cmap = plt.get_cmap('Paired'))

  axes.set_xlabel('lon')
  axes.set_ylabel('lat')
  axes.set_zlabel('rooms')
  plt.show()

### Gráfico mapa

In [ ]:
pos.insert(0,'group',y_kmeans)
pos["group"] = pos["group"].astype(str)
fig = px.scatter_geo(data_frame=pos, lat='latitud', lon='longitud', fitbounds='locations',color='group')
fig.show()
pos = pos.drop('group', axis=1)

#**3. Clasificación**

##a) Construcción del target
Para esta tarea se debe crear una nueva variable tipo_precio que tendrá tres categorías:
alto, medio, bajo. Esta nueva variable será nuestra clase en el problema de clasificación.
Para determinar cuándo el tipo_precio de una propiedad es alto, medio o bajo se deberá
analizar el precio por metro cuadrado (pxm2). Se propone evaluar las siguientes
alternativas para establecer los límites de cada categoría:

In [ ]:
algorithm_pmx2 = lambda row: (row.property_price / row.property_surface_total)
pmx2 = with_target.apply(algorithm_pmx2, axis=1)
pmx2_test = with_target_test.apply(algorithm_pmx2, axis=1)

In [ ]:
prop_caba_data['pxm2'] = pmx2.filter(prop_caba_data.index)
prop_caba_test['pxm2'] = pmx2_test.filter(prop_caba_test.index)

In [ ]:
prop_caba_data.loc[:, ['property_surface_total', 'pxm2']].head()

#### 1. Dividir la variable pxm2 en 3 intervalos con igual cantidad de observaciones.

In [ ]:
#Método para dividir en q cuantiles
def quantile_cut(_row, q, labels=['Bajo', 'Medio', 'Alto']):
  return pd.qcut(_row, q, labels=labels, retbins=True, precision = 5)

In [ ]:
prop_caba_data['tipo_precio'], bins = quantile_cut(prop_caba_data['pxm2'], 3)
prop_caba_data.loc[:, ['property_type', 'pxm2', 'tipo_precio']].sample(n=5)

Verifico cuantos valores cayeron en cada sección

In [ ]:
prop_caba_data['tipo_precio'].value_counts().sort_index()

**● Mostrar la distribución del precio por metro cuadrado**

In [ ]:
sns_hist=sns.histplot(data=prop_caba_data['pxm2'], bins = 50).set(title="Distribución de precio por metro cuadrado",xlabel="pxm2",ylabel="Frecuencia")

**● Mostrar la distribución del precio por metro cuadrado por tipo de propiedad:**

In [ ]:
ax = sns.violinplot(y="pxm2", x="property_type", data=prop_caba_data, hue='tipo_precio').set(title="Gráfico de violín Precio por metro cuadrado Vs Tipo de propiedad",xlabel='Tipo de propiedad',ylabel='pxm2') 

In [ ]:
pxm2_casa = sns.histplot(data=prop_caba_data.loc[prop_caba_data.property_type == 'Casa', ['pxm2']], label='Casa').set(title="Distribución del Precio por metro cuadrado",
                                                                                                                             xlabel='pxm2',
                                                                                                                             ylabel='Frecuencia') 
plt.legend()

In [ ]:
pxm2_PH = sns.histplot(data=prop_caba_data.loc[prop_caba_data.property_type == 'PH', ['pxm2']], label='PH').set(title="Distribución del Precio por metro cuadrado",
                                                                                                                             xlabel='pxm2',
                                                                                                                             ylabel='Frecuencia') 
plt.legend()

In [ ]:
pxm2_Departamento = sns.histplot(data=prop_caba_data.loc[prop_caba_data.property_type == 'Departamento', ['pxm2']], bins = 50, label='Departamento').set(title="Distribución del Precio por metro cuadrado",
                                                                                                                             xlabel='pxm2',
                                                                                                                             ylabel='Frecuencia') 
plt.legend()

In [ ]:
pxm2_casa = sns.histplot(data=prop_caba_data, x='pxm2', hue='property_type', bins = 50).set(title="Distribución del Precio por metro cuadrado",
                                                                                                                             xlabel='pxm2',
                                                                                                                             ylabel='Frecuencia') 

 **● Mostrar gráficamente la distribución de la nueva variable creada tipo_precio:**

In [ ]:
ds_agrupado_Pclass= prop_caba_data.groupby(['tipo_precio']).count()  #Calculo la frecuencia de cada categoría

eje_x = ds_agrupado_Pclass.index.tolist()   ## Declaramos valores para el eje x
eje_y = ds_agrupado_Pclass['pxm2'].tolist()  ## Declaramos valores para el eje y

sns_bar_plot=sns.barplot(x=eje_x, y=eje_y, data=ds_agrupado_Pclass ,alpha=0.5).set(title="Frecuencia de propiedades según tipo de precio",ylabel='Frecuencia',xlabel='tipo_precio')

**● Mostrar en un mapa de CABA los avisos coloreados por tipo_precio**

In [ ]:
fig = px.scatter_geo(data_frame=prop_caba_data, lat='latitud', lon='longitud', fitbounds='locations',color='tipo_precio')
fig.show()

#### 2. Dividir la variable pxm2 en 3 intervalos, el primero con el 25% de las observaciones, el siguiente con el 50% y el último con el 25% de las observaciones restantes.

In [ ]:
prop_caba_data['tipo_precio'], bins = quantile_cut(prop_caba_data['pxm2'], [0, .25, .75, 1])

prop_caba_data.loc[:, ['property_type', 'pxm2', 'tipo_precio']].sample(n=5)

Verifico cuantos valores cayeron en cada cuantil

In [ ]:
prop_caba_data['tipo_precio'].value_counts().sort_index()

**● Mostrar la distribución del precio por metro cuadrado**

In [ ]:
sns_hist=sns.histplot(data=prop_caba_data['pxm2'], bins = 50).set(title="Distribución de precio por metro cuadrado",xlabel="pxm2",ylabel="Frecuencia")

**● Mostrar la distribución del precio por metro cuadrado por tipo de propiedad:**

In [ ]:
ax = sns.violinplot(y="pxm2", x="property_type", data=prop_caba_data, hue='tipo_precio').set(title="Gráfico de violín Precio por metro cuadrado Vs Tipo de propiedad",xlabel='Tipo de propiedad',ylabel='pxm2') 

In [ ]:
pxm2_casa = sns.histplot(data=prop_caba_data.loc[prop_caba_data.property_type == 'Casa', ['pxm2']], label='Casa').set(title="Distribución del Precio por metro cuadrado",
                                                                                                                             xlabel='pxm2',
                                                                                                                             ylabel='Frecuencia') 
plt.legend()

In [ ]:
pxm2_PH = sns.histplot(data=prop_caba_data.loc[prop_caba_data.property_type == 'PH', ['pxm2']], label='PH').set(title="Distribución del Precio por metro cuadrado",
                                                                                                                             xlabel='pxm2',
                                                                                                                             ylabel='Frecuencia') 
plt.legend()

In [ ]:
pxm2_Departamento = sns.histplot(data=prop_caba_data.loc[prop_caba_data.property_type == 'Departamento', ['pxm2']], bins = 50, label='Departamento').set(title="Distribución del Precio por metro cuadrado",
                                                                                                                             xlabel='pxm2',
                                                                                                                             ylabel='Frecuencia') 
plt.legend()

In [ ]:
pxm2_vs_prop_type = sns.histplot(data=prop_caba_data, x='pxm2', hue='property_type', bins = 50).set(title="Distribución del Precio por metro cuadrado",
                                                                                                                             xlabel='pxm2',
                                                                                                                             ylabel='Frecuencia') 

 **● Mostrar gráficamente la distribución de la nueva variable creada tipo_precio:**

In [ ]:
ds_agrupado_Pclass= prop_caba_data.groupby(['tipo_precio']).count()  #Calculo la frecuencia de cada categoría

eje_x = ds_agrupado_Pclass.index.tolist()   ## Declaramos valores para el eje x
eje_y = ds_agrupado_Pclass['pxm2'].tolist()  ## Declaramos valores para el eje y

sns_bar_plot=sns.barplot(x=eje_x, y=eje_y, data=ds_agrupado_Pclass ,alpha=0.5).set(title="Frecuencia de propiedades según tipo de precio",ylabel='Frecuencia',xlabel='tipo_precio')

**● Mostrar en un mapa de CABA los avisos coloreados por tipo_precio**

In [ ]:
fig = px.scatter_geo(data_frame=prop_caba_data, lat='latitud', lon='longitud', fitbounds='locations',color='tipo_precio')
fig.show()

#### 3. Trabajar la variable pxm2 relativa a cada tipo de propiedad y luego dividirla como en el punto anterior.

In [ ]:
indexer_casa = prop_caba_data[prop_caba_data['property_type'] == 'Casa'].index
indexer_PH = prop_caba_data[prop_caba_data['property_type'] == 'PH'].index
indexer_Departamento = prop_caba_data[prop_caba_data['property_type'] == 'Departamento'].index

In [ ]:
#Distribuyo los cuantiles igual que antes (el primero con el 25% de las observaciones, el siguiente con el 50% 
# y el último con el 25% de las observaciones restantes), pero en este caso relativo al tipo de propiedad

prop_caba_data.loc[indexer_casa, 'tipo_precio'], bins_casa = quantile_cut(prop_caba_data.loc[indexer_casa]['pxm2'], [0, .25, .75, 1])
prop_caba_data.loc[indexer_PH, 'tipo_precio'], bins_PH = quantile_cut(prop_caba_data.loc[indexer_PH]['pxm2'], [0, .25, .75, 1])
prop_caba_data.loc[indexer_Departamento, 'tipo_precio'], bins_Departamento = quantile_cut(prop_caba_data.loc[indexer_Departamento]['pxm2'], [0, .25, .75, 1])

prop_caba_data.loc[:, ['property_type', 'pxm2', 'tipo_precio']].sample(n=5)

Verifico cuantos valores cayeron en cada cuantil

In [ ]:
prop_caba_data['tipo_precio'].value_counts().sort_index()

**● Mostrar la distribución del precio por metro cuadrado**

In [ ]:
sns_hist=sns.histplot(data=prop_caba_data['pxm2'], bins = 50).set(title="Distribución de precio por metro cuadrado",xlabel="pxm2",ylabel="Frecuencia")

**● Mostrar la distribución del precio por metro cuadrado por tipo de propiedad:**

In [ ]:
ax = sns.violinplot(y="pxm2", x="property_type", data=prop_caba_data, hue='tipo_precio').set(title="Gráfico de violín Precio por metro cuadrado Vs Tipo de propiedad",xlabel='Tipo de propiedad',ylabel='pxm2')

In [ ]:
pxm2_casa = sns.histplot(data=prop_caba_data.loc[prop_caba_data.property_type == 'Casa', ['pxm2']], label='Casa').set(title="Distribución del Precio por metro cuadrado",
                                                                                                                             xlabel='pxm2',
                                                                                                                             ylabel='Frecuencia') 
plt.legend()

In [ ]:
pxm2_PH = sns.histplot(data=prop_caba_data.loc[prop_caba_data.property_type == 'PH', ['pxm2']], label='PH').set(title="Distribución del Precio por metro cuadrado",
                                                                                                                             xlabel='pxm2',
                                                                                                                             ylabel='Frecuencia') 
plt.legend()

In [ ]:
pxm2_Departamento = sns.histplot(data=prop_caba_data.loc[prop_caba_data.property_type == 'Departamento', ['pxm2']], bins = 50, label='Departamento').set(title="Distribución del Precio por metro cuadrado",
                                                                                                                             xlabel='pxm2',
                                                                                                                             ylabel='Frecuencia') 
plt.legend()

In [ ]:
pxm2_vs_prop_type = sns.histplot(data=prop_caba_data, x='pxm2', hue='property_type', bins = 50).set(title="Distribución del Precio por metro cuadrado",
                                                                                                                             xlabel='pxm2',
                                                                                                                             ylabel='Frecuencia') 

 **● Mostrar gráficamente la distribución de la nueva variable creada tipo_precio:**

In [ ]:
ds_agrupado_Pclass= prop_caba_data.groupby(['tipo_precio']).count()  #Calculo la frecuencia de cada categoría

eje_x = ds_agrupado_Pclass.index.tolist()   ## Declaramos valores para el eje x
eje_y = ds_agrupado_Pclass['pxm2'].tolist()  ## Declaramos valores para el eje y

sns_bar_plot=sns.barplot(x=eje_x, y=eje_y, data=ds_agrupado_Pclass ,alpha=0.5).set(title="Frecuencia de propiedades según tipo de precio",ylabel='Frecuencia',xlabel='tipo_precio')

**● Mostrar en un mapa de CABA los avisos coloreados por tipo_precio**

In [ ]:
fig = px.scatter_geo(data_frame=prop_caba_data, lat='latitud', lon='longitud', fitbounds='locations',color='tipo_precio')
fig.show()

### Escoger una de las opciones
Se toma la decisión de utilizar la opción 3, pues resulta apropiado adaptar la clasificación al tipo de propiedad. Lógicamente, un apartamento de precio alto y una casa de precio alto no cuestan lo mismo, tal y como se puede apreciar a continuación haciendo un análisis de los intervalos obtenidos para la clasificación:

In [ ]:
bins_casa = list(np.float_(bins_casa))
print(bins_casa)

In [ ]:
bins_PH = list(np.float_(bins_PH))
print(bins_PH)

In [ ]:
bins_Departamento = list(np.float_(bins_Departamento))
print(bins_Departamento)

### A partir de los bins obtenidos en train, se replica la discretización para test

In [ ]:
indexer_casa = prop_caba_test[prop_caba_test['property_type'] == 'Casa'].index
indexer_PH = prop_caba_test[prop_caba_test['property_type'] == 'PH'].index
indexer_Departamento = prop_caba_test[prop_caba_test['property_type'] == 'Departamento'].index

In [ ]:
indexer_casa

In [ ]:
pmx2_test

In [ ]:
prop_caba_test

In [ ]:
#Distribuyo el tipo_precio relativo al tipo de propiedad, A PARTIR DE LOS BINS OBTENIDOS EN TRAIN!!
bins_casa[0] = 0
bins_casa[len(bins_casa) - 1] = 10000

bins_PH[0] = 0
bins_PH[len(bins_PH) - 1] = 10000

bins_Departamento[0] = 0
bins_Departamento[len(bins_Departamento) - 1] = 10000

prop_caba_test.loc[indexer_casa, 'tipo_precio'] = pd.cut(prop_caba_test.loc[indexer_casa]['pxm2'], bins = bins_casa, labels = ['Bajo', 'Medio', 'Alto'])
prop_caba_test.loc[indexer_PH, 'tipo_precio'] = pd.cut(prop_caba_test.loc[indexer_PH]['pxm2'], bins = bins_PH, labels = ['Bajo', 'Medio', 'Alto'])
prop_caba_test.loc[indexer_Departamento, 'tipo_precio'] = pd.cut(prop_caba_test.loc[indexer_Departamento]['pxm2'], bins_Departamento, labels = ['Bajo', 'Medio', 'Alto'])
prop_caba_test.loc[:, ['property_type', 'pxm2', 'tipo_precio']].sample(n=5)

In [ ]:
prop_caba_test.loc[prop_caba_test.tipo_precio == np.NaN]

### Se guardan los conjuntos de datos obtenidos luego de realizar la construcción del target:

In [ ]:
df_clasificacion_train = prop_caba_data.copy()
df_clasificacion_train['property_price'] = with_target.property_price
df_clasificacion_train.head(2)

In [ ]:
df_clasificacion_test = prop_caba_test.copy()
df_clasificacion_test['property_price'] = with_target_test.property_price
df_clasificacion_test.head(2)

In [ ]:
try:
  df_clasificacion_train.to_csv('drive/My Drive/tp1 datos grupal/Datasets/df_clasificacion_train.csv', index_label="index")
  df_clasificacion_test.to_csv('drive/My Drive/tp1 datos grupal/Datasets/df_clasificacion_test.csv', index_label="index")
except:
  df_clasificacion_train.to_csv('df_clasificacion_train.csv', index_label="index")
  df_clasificacion_test.to_csv('df_clasificacion_test.csv', index_label="index")

##b) Entrenamiento y Predicción

Predecir el valor del atributo tipo_precio, excluyendo del entrenamiento la variable price,
pxm2 y cualquier otra que pueda contener información del precio de venta. Para todos los
modelos se pide:

###● Realizar las tareas de ingeniería de características necesarias para trabajar con cada algoritmo (encoding, normalización, etc)

Mapeamos los valores `tipo_precio` a variables numéricas para poder procesarlos

In [ ]:
prop_caba_data['tipo_precio'] = prop_caba_data['tipo_precio'].map({'Bajo':0,'Medio':1, 'Alto':2},na_action=None)
prop_caba_test['tipo_precio'] = prop_caba_test['tipo_precio'].map({'Bajo':0,'Medio':1, 'Alto':2},na_action=None)

Antes de realizar el análisis de hiperparámetros, verificamos que las clases estén correctamente balanceadas

In [ ]:
features=['latitud', 'longitud', 'property_rooms', 'property_bedrooms', 'property_surface_total', 
          'property_surface_covered', 'property_type_Departamento', 'property_type_PH',
          'comuna_2', 'comuna_3', 'comuna_4', 'comuna_5', 'comuna_6', 'comuna_7', 'comuna_8',
          'comuna_9', 'comuna_10', 'comuna_11', 'comuna_12', 'comuna_13', 'comuna_14', 'comuna_15']

target='tipo_precio'

#Verifico balanceo de clases
prop_caba_data[target].value_counts(normalize=True)*100

### Se guardan los conjuntos de datos obtenidos luego de realizar ingenieria de características:

#### DataFrame original

In [ ]:
df_encoded_train.head(2)

In [ ]:
df_clasificacion_train_ENCODED = df_encoded_train.copy()
df_clasificacion_train_ENCODED['tipo_precio'] = prop_caba_data.tipo_precio
df_clasificacion_train_ENCODED['pxm2'] = prop_caba_data.pxm2
df_clasificacion_train_ENCODED.head(2)

In [ ]:
df_clasificacion_test_ENCODED = df_encoded_test.copy()
df_clasificacion_test_ENCODED['tipo_precio'] = prop_caba_test.tipo_precio
df_clasificacion_test_ENCODED['pxm2'] = prop_caba_test.pxm2
df_clasificacion_test_ENCODED.head(2)

In [ ]:
try:
  df_clasificacion_train_ENCODED.to_csv('drive/My Drive/tp1 datos grupal/Datasets/df_clasificacion_train_ENCODED.csv', index_label="index")
  df_clasificacion_test_ENCODED.to_csv('drive/My Drive/tp1 datos grupal/Datasets/df_clasificacion_test_ENCODED.csv', index_label="index")
except:
  df_clasificacion_train_ENCODED.to_csv('df_clasificacion_train_ENCODED.csv', index_label="index")
  df_clasificacion_test_ENCODED.to_csv('df_clasificacion_test_ENCODED.csv', index_label="index")

#### DataFrame PCA

In [ ]:
df_PCA_ENCODED_train.head(2)

In [ ]:
df_clasificacion_PCA_train_ENCODED = df_PCA_ENCODED_train.copy()
df_clasificacion_PCA_train_ENCODED['tipo_precio'] = prop_caba_data.tipo_precio
df_clasificacion_PCA_train_ENCODED['pxm2'] = prop_caba_data.pxm2
df_clasificacion_PCA_train_ENCODED.head(2)

In [ ]:
df_clasificacion_PCA_test_ENCODED = df_PCA_ENCODED_test.copy()
df_clasificacion_PCA_test_ENCODED['tipo_precio'] = prop_caba_test.tipo_precio
df_clasificacion_PCA_test_ENCODED['pxm2'] = prop_caba_test.pxm2
df_clasificacion_PCA_test_ENCODED.head(2)

In [ ]:
try:
  df_clasificacion_PCA_train_ENCODED.to_csv('drive/My Drive/tp1 datos grupal/Datasets/df_clasificacion_PCA_train_ENCODED.csv', index_label="index")
  df_clasificacion_PCA_test_ENCODED.to_csv('drive/My Drive/tp1 datos grupal/Datasets/df_clasificacion_PCA_test_ENCODED.csv', index_label="index")
except:
  df_clasificacion_PCA_train_ENCODED.to_csv('df_clasificacion_PCA_train_ENCODED.csv', index_label="index")
  df_clasificacion_PCA_test_ENCODED.to_csv('df_clasificacion_PCA_test_ENCODED.csv', index_label="index")

## Arbol de decisión

A continuación, separamos el set de entrenamiento y pruebas y verificamos que los datos se encuentren correctamente balanceados

### Dataset

In [ ]:
#Separo un set de Evaluacion
from sklearn.model_selection import train_test_split
decision_tree_data = df_encoded_train.drop('property_price', axis=1)
x_train = decision_tree_data.values
y_train =  prop_caba_data[target].values

features = decision_tree_data.columns

decision_tree_test = df_encoded_test.drop('property_price', axis=1)
x_test = decision_tree_test.values
y_test =  prop_caba_test[target].values

#Verifico Cantidad de Datos en cada set
print('# Datos Entrenamiento: {}'.format(len(x_train)))
print('# Datos Prueba: {}'.format(len(x_test)))

#Verifico como quedaron balanceados
for split_name, split in zip(['% Positivos Entrenamiento','% Positivos Prueba'],[y_train,y_test]):
  print('{}: {:.3f}'.format(split_name,pd.Series(split).value_counts(normalize=True)[0]*100), 'Bajo')
  print('{}: {:.3f}'.format(split_name,pd.Series(split).value_counts(normalize=True)[1]*100), 'Medio')
  print('{}: {:.3f}'.format(split_name,pd.Series(split).value_counts(normalize=True)[2]*100), 'Alto')

Aplicamos Cross-Validation y para esto, seteamos 20 combinaciones posibles con 10 folds ya que, durante previos testeos encontramos que para entretar con CV un arbol de decisión, se realiza relativamente rápido (en el rango de 1 a 2 minutos).

Por otro lado, nos enfocamos de optimizar el f1-score ya que queremos dar una buena predicción tanto para el recall como también para el accuracy. 

#### Grid search

In [ ]:
import sklearn as sk
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer

#Cantidad de combinaciones que quiero porbar
n=3

#Conjunto de parámetros que quiero usar
params_grid = {'criterion':['gini','entropy'],
               'ccp_alpha':np.linspace(0,0.05,n), 
               'max_depth':list(range(1,10,2))}
                
#Cantidad de splits para el Cross Validation
folds=10

#Kfold estratificado
kfoldcv = StratifiedKFold(n_splits=folds)

#Clasificador
base_tree = DecisionTreeClassifier()

#Metrica que quiero optimizar F1 Score
scorer_fn = make_scorer(sk.metrics.f1_score, average='micro')

#GridSearch CV
gridcv = GridSearchCV(estimator=base_tree ,param_grid=params_grid,scoring=scorer_fn,cv=kfoldcv,return_train_score='True')

gridcv.fit(x_train,y_train)

In [ ]:
#Mejores hiperparametros del arbol
print(gridcv.best_params_)
#Mejor métrica
print(gridcv.best_score_)

#### Importancia

Ahora validamos los atributos con mayor importancia y podemos apreciar de que, en este caso corresponden a Latitud y Longitud. Luego, para este contexto, tiene sentido decir que el precio tenga relación con estos atributos ya que los mismos corresponden a la geolocalización de las propiedades y para el caso de Capital Federal, los precios varían según el barrio.

In [ ]:
#Atributos considerados y su importancia
best_tree = gridcv.best_estimator_
feat_imps = best_tree.feature_importances_

for feat_imp,feat in sorted(zip(feat_imps,features)):
  if feat_imp>0:
    print('{}: {}'.format(feat,feat_imp))

#### Mejor modelo

In [ ]:
#Creo el árbol con los mejores hiperparámetros
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text

clf=DecisionTreeClassifier().set_params(**gridcv.best_params_)

#Entreno el arbol en todo el set
clf.fit(x_train,y_train)

rules = export_text(clf, feature_names=list(features))

In [ ]:
from six import StringIO
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
import matplotlib.pyplot as plt

dot_data = StringIO()
export_graphviz(max_depth = 3, clf, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,
                feature_names=features,
                class_names=['Bajo', 'Medio', 'Alto'])

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png())

#### Evaluar

Evaluando el Arbol con los mejores hiperparámetros en train:

In [ ]:
#Evalúo el Arbol con los mejores hiperparámetros
from sklearn.metrics import confusion_matrix, classification_report , f1_score

#Hago predicción sobre el set de evaluacion
y_pred= clf.predict(x_train)

#Arbol Reporte y Matriz de Confusion
#print(classification_report(y_test,y_pred))
print('F1-Score: {}'.format(f1_score(y_train, y_pred, average='micro')))

cm = confusion_matrix(y_train,y_pred)

# Creating a dataframe for a array-formatted Confusion matrix,so it will be easy for plotting.
cm_df = pd.DataFrame(cm,
                     index = ['Alto','Medio','Bajo'], 
                     columns = ['Alto','Medio','Bajo'])

#Plotting the confusion matrix
plt.figure(figsize=(10,8))
sns.heatmap(cm_df, cmap='Blues', annot=True, fmt='g')
plt.title('Confusion Matrix')
plt.ylabel('Actual Values')
plt.xlabel('Predicted Values')
plt.show()

En primera instancia, vemos que hay muchos mas aciertos entre los precios alto-bajo en relación a los alto-medio o medio-bajo

In [ ]:
#Evalúo el Arbol con los mejores hiperparámetros
from sklearn.metrics import confusion_matrix, classification_report , f1_score, recall_score, accuracy_score

#Hago predicción sobre el set de evaluacion
y_pred= clf.predict(x_test)

#Arbol Reporte y Matriz de Confusion
#print(classification_report(y_test,y_pred))
print('Accuracy: {}'.format(accuracy_score(y_test, y_pred)))
print('Recall: {}'.format(recall_score(y_test, y_pred, average='micro')))
print('F1-Score: {}'.format(f1_score(y_test, y_pred, average='micro')))

cm = confusion_matrix(y_test,y_pred)

# Creating a dataframe for a array-formatted Confusion matrix,so it will be easy for plotting.
cm_df = pd.DataFrame(cm,
                     index = ['Alto','Medio','Bajo'], 
                     columns = ['Alto','Medio','Bajo'])

#Plotting the confusion matrix
plt.figure(figsize=(10,8))
sns.heatmap(cm_df, cmap='Blues', annot=True, fmt='g')
plt.title('Confusion Matrix')
plt.ylabel('Actual Values')
plt.xlabel('Predicted Values')
plt.show()

En base al f1 score obtenido, el modelo performa de manera muy similar en train y en test.

Analizando los resultados, vemos que las predicciones entre los valores bajo y alto son muy acertadas. Esto es ya que en la mayoria de los datos hay mucha diferencia entre los precios que consideramos altos y bajos.

Por otro lado, si analizamos las predicciones entre medio y alto o medio y bajo, tenemos muchos menos aciertos. Por este motivo, es que priorizamos anteriormente la metrica f-score para poder enfocarnos tanto en la precisión como en el recall.

Se guarda el modelo de arbol de decisión:

In [ ]:
import joblib  

try:
  joblib.dump(clf, 'drive/My Drive/tp1 datos grupal/Modelos/ArbolDeDecision.pkl')
except:
  joblib.dump(clf, 'ArbolDeDecision.pkl')

### PCA

A continuación, separamos el set de entrenamiento y pruebas y verificamos que los datos se encuentren correctamente balanceados

In [ ]:
df_PCA_ENCODED_train

####Dataset

In [ ]:
#Separo un set de Evaluacion
from sklearn.model_selection import train_test_split
decision_tree_data_PCA = df_PCA_ENCODED_train.drop('property_price', axis=1)
x_train_PCA = decision_tree_data_PCA.values
y_train_PCA =  prop_caba_data[target].values

features_PCA = decision_tree_data_PCA.columns

decision_tree_test_PCA = df_PCA_ENCODED_test.drop('property_price', axis=1)
x_test_PCA = decision_tree_test_PCA.values
y_test_PCA =  prop_caba_test[target].values

#Verifico Cantidad de Datos en cada set
print('# Datos Entrenamiento: {}'.format(len(x_train_PCA)))
print('# Datos Prueba: {}'.format(len(x_test_PCA)))

#Verifico como quedaron balanceados
for split_name, split in zip(['% Positivos Entrenamiento','% Positivos Prueba'],[y_train_PCA,y_test_PCA]):
  print('{}: {:.3f}'.format(split_name,pd.Series(split).value_counts(normalize=True)[0]*100), 'Bajo')
  print('{}: {:.3f}'.format(split_name,pd.Series(split).value_counts(normalize=True)[1]*100), 'Medio')
  print('{}: {:.3f}'.format(split_name,pd.Series(split).value_counts(normalize=True)[2]*100), 'Alto')

#### Grid search

In [ ]:
import sklearn as sk
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer

#Cantidad de combinaciones que quiero porbar
n=3

#Conjunto de parámetros que quiero usar
params_grid = {'criterion':['gini','entropy'],
               'ccp_alpha':np.linspace(0,0.05,n), 
               'max_depth':list(range(1,10,2))}
                
#Cantidad de splits para el Cross Validation
folds=10

#Kfold estratificado
kfoldcv = StratifiedKFold(n_splits=folds)

#Clasificador
base_tree = DecisionTreeClassifier()

#Metrica que quiero optimizar F1 Score
scorer_fn = make_scorer(sk.metrics.f1_score, average='micro')

#GridSearch CV
gridcv = GridSearchCV(estimator=base_tree ,param_grid=params_grid,scoring=scorer_fn,cv=kfoldcv,return_train_score='True')

gridcv.fit(x_train_PCA,y_train_PCA)

In [ ]:
#Mejores hiperparametros del arbol
print(gridcv.best_params_)
#Mejor métrica
print(gridcv.best_score_)

#### Importancia

Ahora validamos los atributos con mayor importancia y podemos apreciar de que, en este caso corresponden a las 3 componentes principales, sobre todo a la pc_2, que como vimos anteriormente es la que más peso le da a la latitud y longitud de la propiedad. Luego, para este contexto, tiene sentido decir que el precio tenga relación con estos atributos ya que los mismos corresponden a la geolocalización de las propiedades y para el caso de Capital Federal, los precios varían según el barrio.

In [ ]:
#Atributos considerados y su importancia
best_tree = gridcv.best_estimator_
feat_imps = best_tree.feature_importances_

for feat_imp,feat in sorted(zip(feat_imps,features_PCA)):
  if feat_imp>0:
    print('{}: {}'.format(feat,feat_imp))

#### Mejor modelo

In [ ]:
#Creo el árbol con los mejores hiperparámetros
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text

tree_PCA = DecisionTreeClassifier().set_params(**gridcv.best_params_)

#Entreno el arbol en todo el set
tree_PCA.fit(x_train_PCA,y_train_PCA)

rules = export_text(tree_PCA, feature_names=list(features_PCA))

In [ ]:
from six import StringIO
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
import matplotlib.pyplot as plt

dot_data = StringIO()
export_graphviz(max_depth = 3, tree_PCA, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,
                feature_names=features_PCA,
                class_names=['Bajo', 'Medio', 'Alto'])

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

#### Evaluar

Se evalúa el modelo entrenado con PCA en train

In [ ]:
#Evalúo el Arbol con los mejores hiperparámetros
from sklearn.metrics import confusion_matrix, classification_report , f1_score

#Hago predicción sobre el set de evaluacion
y_pred_PCA= tree_PCA.predict(x_train_PCA)

#Arbol Reporte y Matriz de Confusion
#print(classification_report(y_test,y_pred))
print('F1-Score: {}'.format(f1_score(y_train_PCA, y_pred_PCA, average='micro')))

cm = confusion_matrix(y_train_PCA,y_pred_PCA)

# Creating a dataframe for a array-formatted Confusion matrix,so it will be easy for plotting.
cm_df = pd.DataFrame(cm,
                     index = ['Alto','Medio','Bajo'], 
                     columns = ['Alto','Medio','Bajo'])

#Plotting the confusion matrix
plt.figure(figsize=(10,8))
sns.heatmap(cm_df, cmap='Blues', annot=True, fmt='g')
plt.title('Confusion Matrix')
plt.ylabel('Actual Values')
plt.xlabel('Predicted Values')
plt.show()

Dado el valor de f1-score obtenido vemos que performa de manera similar a el modelo entrenado sin realizar la reducción de dimensión, se analiza a continuación lo obtenido en test:

In [ ]:
#Evalúo el Arbol con los mejores hiperparámetros
from sklearn.metrics import confusion_matrix, classification_report , f1_score

#Hago predicción sobre el set de evaluacion
y_pred_PCA= tree_PCA.predict(x_test_PCA)

#Arbol Reporte y Matriz de Confusion
#print(classification_report(y_test,y_pred))
print('F1-Score: {}'.format(f1_score(y_test_PCA, y_pred_PCA, average='micro')))

cm = confusion_matrix(y_test_PCA,y_pred_PCA)

# Creating a dataframe for a array-formatted Confusion matrix,so it will be easy for plotting.
cm_df = pd.DataFrame(cm,
                     index = ['Alto','Medio','Bajo'], 
                     columns = ['Alto','Medio','Bajo'])

#Plotting the confusion matrix
plt.figure(figsize=(10,8))
sns.heatmap(cm_df, cmap='Blues', annot=True, fmt='g')
plt.title('Confusion Matrix')
plt.ylabel('Actual Values')
plt.xlabel('Predicted Values')
plt.show()

Esta vez vemos que el valor de f1-score es el que peor performa con los datos de test. 

Nuevamente performó de forma similar en train y en test, logrando resultados comparables a los obtenidos entrenando el modelo con el dataset completo.

Se guarda el modelo de arbol de decisión entrenado usando PCA:

In [ ]:
import joblib  

try:
  joblib.dump(tree_PCA, 'drive/My Drive/tp1 datos grupal/Modelos/ArbolDeDecision_PCA.pkl')
except:
  joblib.dump(tree_PCA, 'ArbolDeDecision_PCA.pkl')

## Random forest

###Dataset


#### Grid search

Para el Random Forest, queremos nuevamente optimizar el f1-score ya que necesitamos poder dar una buena predicción con un buen accuracy y un buen recall

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#Cantidad de combinaciones que quiero porbar
n=3

#Conjunto de parámetros que quiero usar
params_grid = {'criterion':['gini','entropy'],
               'n_estimators':[4,6,8,10],
               'ccp_alpha':np.linspace(0,0.05,n), 
               'max_depth':list(range(1,6))}
                
#Cantidad de splits para el Cross Validation
folds=5

#Kfold estratificado
kfoldcv = StratifiedKFold(n_splits=folds)

#Clasificador
rf_tree = RandomForestClassifier()

#Metrica que quiero optimizar F1 Score
scorer_fn = make_scorer(sk.metrics.f1_score, average='micro')

#GridSearch CV
gridcv = GridSearchCV(estimator=rf_tree ,param_grid=params_grid,scoring=scorer_fn,cv=kfoldcv,return_train_score='True')

gridcv.fit(x_train,y_train)

In [ ]:
#Mejores hiperparametros del arbol
print(gridcv.best_params_)
#Mejor métrica
print(gridcv.best_score_)

#### Importancia

In [ ]:
#Atributos considerados y su importancia
best_tree = gridcv.best_estimator_
feat_imps = best_tree.feature_importances_

for feat_imp,feat in sorted(zip(feat_imps,features)):
  if feat_imp>0:
    print('{}: {}'.format(feat,feat_imp))

En esta caso, vemos que nuevamente que la latitud y longitud son uno de los atributos más importante ya que indica la geoposición de la vivienda. Esto tiene sentido ya que, sin analizar los datos, podíamos estimar que el precio de una propiedad tiene una relación con su posición geográfica. 

Por otro lado, otro de los atributos importantes que vemos es la superficie cubierta y la superficie total que también son puntos que sabíamos que iban a tener una gran relación.

#### Mejor modelo

In [ ]:
rf_tree_clf = RandomForestClassifier().set_params(**gridcv.best_params_)

#Entreno el arbol en todo el set
rf_tree_clf.fit(x_train,y_train)

In [ ]:
dot_data = StringIO()

#Grafico uno de los arboles
export_graphviz(max_depth = 3, rf_tree_clf.estimators_[0], out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,
                feature_names=features,
                class_names=['Alto','Medio', 'Bajo'])

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

#### Evaluar

Se evalúa el modelo para datos de train:

In [ ]:
#Hago predicción sobre el set de evaluacion
y_pred= rf_tree_clf.predict(x_train)

#Arbol Reporte y Matriz de Confusion
#print(classification_report(y_test,y_pred))
print('F1-Score: {}'.format(f1_score(y_train, y_pred, average='micro')))

cm = confusion_matrix(y_train,y_pred)

# Creating a dataframe for a array-formatted Confusion matrix,so it will be easy for plotting.
cm_df = pd.DataFrame(cm,
                     index = ['Alto','Medio','Bajo'], 
                     columns = ['Alto','Medio','Bajo'])

#Plotting the confusion matrix
plt.figure(figsize=(10,8))
sns.heatmap(cm_df, cmap='Blues', annot=True, fmt='g')
plt.title('Confusion Matrix')
plt.ylabel('Actual Values')
plt.xlabel('Predicted Values')
plt.show()

El F1 score obtenido para datos de train es similar a los modelos anteriores, en la matriz de confusión se puede ver como el modelo predice la mayoria de las veces precio 'Medio'. a continuación sanaliza la performance ahora para datos de test:

In [ ]:
#Hago predicción sobre el set de evaluacion
y_pred= rf_tree_clf.predict(x_test)

#Arbol Reporte y Matriz de Confusion
#print(classification_report(y_test,y_pred))
print('F1-Score: {}'.format(f1_score(y_test, y_pred, average='micro')))

cm = confusion_matrix(y_test,y_pred)

# Creating a dataframe for a array-formatted Confusion matrix,so it will be easy for plotting.
cm_df = pd.DataFrame(cm,
                     index = ['Alto','Medio','Bajo'], 
                     columns = ['Alto','Medio','Bajo'])

#Plotting the confusion matrix
plt.figure(figsize=(10,8))
sns.heatmap(cm_df, cmap='Blues', annot=True, fmt='g')
plt.title('Confusion Matrix')
plt.ylabel('Actual Values')
plt.xlabel('Predicted Values')
plt.show()

El resultado obtenido es similar al de test.

Se guarda el modelo Random Forest:

In [ ]:
import joblib  

try:
  joblib.dump(rf_tree_clf, 'drive/My Drive/tp1 datos grupal/Modelos/RandomForest.pkl')
except:
  joblib.dump(rf_tree_clf, 'RandomForest.pkl')

### PCA

Se replica a continuación el análisis pero utilizando el data set con la reducción de dimensionalidad aplicada en el punto 1)d.

#### Grid search

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#Cantidad de combinaciones que quiero porbar
n=3

#Conjunto de parámetros que quiero usar
params_grid = {'criterion':['gini','entropy'],
               'n_estimators':[4,6,8,10],
               'ccp_alpha':np.linspace(0,0.05,n), 
               'max_depth':list(range(1,6))}
                
#Cantidad de splits para el Cross Validation
folds=5

#Kfold estratificado
kfoldcv = StratifiedKFold(n_splits=folds)

#Clasificador
rf_tree = RandomForestClassifier()

#Metrica que quiero optimizar F1 Score
scorer_fn = make_scorer(sk.metrics.f1_score, average='micro')

#GridSearch CV
gridcv = GridSearchCV(estimator=rf_tree ,param_grid=params_grid,scoring=scorer_fn,cv=kfoldcv,return_train_score='True')

gridcv.fit(x_train_PCA, y_train_PCA)

In [ ]:
#Mejores hiperparametros del arbol
print(gridcv.best_params_)
#Mejor métrica
print(gridcv.best_score_)

#### Importancia

In [ ]:
#Atributos considerados y su importancia
best_tree = gridcv.best_estimator_
feat_imps = best_tree.feature_importances_

for feat_imp,feat in sorted(zip(feat_imps,features_PCA)):
  if feat_imp>0:
    print('{}: {}'.format(feat,feat_imp))

#### Mejor modelo

In [ ]:
rf_tree_clf_PCA = RandomForestClassifier().set_params(**gridcv.best_params_)

#Entreno el arbol en todo el set
rf_tree_clf_PCA.fit(x_train_PCA,y_train_PCA)

In [ ]:
dot_data = StringIO()

#Grafico uno de los arboles
export_graphviz(max_depth = 3, rf_tree_clf_PCA.estimators_[0], out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,
                feature_names=features_PCA,
                class_names=['Alto','Medio', 'Bajo'])

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

#### Evaluar

Se evalúa la performance del modelo para datos de train

In [ ]:
y_pred_PCA= rf_tree_clf_PCA.predict(x_train_PCA)

#Arbol Reporte y Matriz de Confusion
#print(classification_report(y_test,y_pred))
print('F1-Score: {}'.format(f1_score(y_train_PCA, y_pred_PCA, average='micro')))

cm = confusion_matrix(y_train_PCA,y_pred_PCA)

# Creating a dataframe for a array-formatted Confusion matrix,so it will be easy for plotting.
cm_df = pd.DataFrame(cm,
                     index = ['Alto','Medio','Bajo'], 
                     columns = ['Alto','Medio','Bajo'])

#Plotting the confusion matrix
plt.figure(figsize=(10,8))
sns.heatmap(cm_df, cmap='Blues', annot=True, fmt='g')
plt.title('Confusion Matrix')
plt.ylabel('Actual Values')
plt.xlabel('Predicted Values')
plt.show()

A continuación se evalúa para datos de test

In [ ]:
#Hago predicción sobre el set de evaluacion
y_pred_PCA= rf_tree_clf_PCA.predict(x_test_PCA)

#Arbol Reporte y Matriz de Confusion
#print(classification_report(y_test,y_pred))
print('F1-Score: {}'.format(f1_score(y_test_PCA, y_pred_PCA, average='micro')))

cm = confusion_matrix(y_test_PCA,y_pred_PCA)

# Creating a dataframe for a array-formatted Confusion matrix,so it will be easy for plotting.
cm_df = pd.DataFrame(cm,
                     index = ['Alto','Medio','Bajo'], 
                     columns = ['Alto','Medio','Bajo'])

#Plotting the confusion matrix
plt.figure(figsize=(10,8))
sns.heatmap(cm_df, cmap='Blues', annot=True, fmt='g')
plt.title('Confusion Matrix')
plt.ylabel('Actual Values')
plt.xlabel('Predicted Values')
plt.show()

Nuevamente las performance en train y test resultan similares, y el modelo predice la mayoría de las veces tipo_precio Medio.

Se guarda el modelo Random Forest entrenado con PCA

In [ ]:
import joblib  

try:
  joblib.dump(rf_tree_clf_PCA, 'drive/My Drive/tp1 datos grupal/Modelos/RandomForest_PCA.pkl')
except:
  joblib.dump(rf_tree_clf_PCA, 'RandomForest_PCA.pkl')

## CatBoost Classifier

### Dataset original

In [ ]:
!pip install catboost
import catboost as cb

In [ ]:
import numpy as np

x_train = df_encoded_train.drop('property_price', axis=1).values
y_train = prop_caba_data[target].tolist()


x_test = df_encoded_test.drop('property_price', axis=1).values
y_test =  prop_caba_test[target].tolist()

model =  cb.CatBoostClassifier(random_seed = 0, loss_function='MultiClass', verbose=False)

grid = {'learning_rate': [0.7, 0.8, 0.9],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1,10,20]}

grid_search_result = model.grid_search(grid, 
                                       X=x_train[:60], 
                                       y=y_train[:60])

In [ ]:
grid_search_result['params']

In [ ]:
model = cb.CatBoostClassifier(random_seed = 0, l2_leaf_reg = grid_search_result['params']['l2_leaf_reg'],
                          learning_rate=grid_search_result['params']['learning_rate'],
                          depth=grid_search_result['params']['depth'])
# Fit model
model.fit(x_train, y_train)
# Get predictions for train
y_pred = model.predict(x_train)

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score
print('F1-Score: {}'.format(f1_score(y_train, y_pred, average='micro')))

cm = confusion_matrix(prop_caba_data[target].tolist(),y_pred)

# Creating a dataframe for a array-formatted Confusion matrix,so it will be easy for plotting.
cm_df = pd.DataFrame(cm,
                     index = ['Alto','Medio','Bajo'], 
                     columns = ['Alto','Medio','Bajo'])

#Plotting the confusion matrix
plt.figure(figsize=(10,8))
sns.heatmap(cm_df, cmap='Blues', annot=True, fmt='g')
plt.title('Confusion Matrix')
plt.ylabel('Actual Values')
plt.xlabel('Predicted Values')
plt.show()

Para train se obtiene un F1 score alto. Analizamos los resultados en test:

In [ ]:
# Get predictions
y_pred = model.predict(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score
print('F1-Score: {}'.format(f1_score(y_test, y_pred, average='micro')))

cm = confusion_matrix(prop_caba_test[target].tolist(),y_pred)

# Creating a dataframe for a array-formatted Confusion matrix,so it will be easy for plotting.
cm_df = pd.DataFrame(cm,
                     index = ['Alto','Medio','Bajo'], 
                     columns = ['Alto','Medio','Bajo'])

#Plotting the confusion matrix
plt.figure(figsize=(10,8))
sns.heatmap(cm_df, cmap='Blues', annot=True, fmt='g')
plt.title('Confusion Matrix')
plt.ylabel('Actual Values')
plt.xlabel('Predicted Values')
plt.show()

Al igual que en los modelos anteriores, decidimos utilizar la métrica del f1-score para poder tener un mejor resultado en predicciones para el accuracy y el recall.

En este caso obtuvimos para test un F1 de aproximadamente 0.7, resultando entonces en nuestro mejor modelo de clasificación. A que costo? El tiempo de ejecución es muy elevado, la búsqueda de hiperparámetros se realiza con porciones muy pequeñas de datos, de modo contrario se cuelga y finaliza la ejecución.

Se guarda el modelo Cat Boost

In [ ]:
import joblib  

try:
  joblib.dump(model, 'drive/My Drive/tp1 datos grupal/Modelos/CatBoostClassifier.pkl')
except:
  joblib.dump(model, 'CatBoostClassifier.pkl')

### PCA

In [ ]:
#Separo un set de Evaluacion
y_train = prop_caba_data[target].tolist()
y_test =  prop_caba_test[target].tolist()

X_train = df_PCA_ENCODED_train.drop('property_price', axis=1)
X_test = df_PCA_ENCODED_test.drop('property_price', axis=1)

#Verifico Cantidad de Datos en cada set
print('# Datos Entrenamiento: {}'.format(len(x_train)))
print('# Datos Prueba: {}'.format(len(x_test)))


In [ ]:
model =   cb.CatBoostClassifier(random_seed = 0, verbose=False)

grid = {'learning_rate': [0.7, 0.8, 0.9],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1,10,20]}

grid_search_result = model.grid_search(grid, 
                                       X=X_train[:60], 
                                       y=y_train[:60], verbose=False)

In [ ]:
model = cb.CatBoostClassifier(l2_leaf_reg = grid_search_result['params']['l2_leaf_reg'],
                          learning_rate=grid_search_result['params']['learning_rate'],
                          depth=grid_search_result['params']['depth'], verbose=False, random_seed = 0)
# Fit model
model.fit(X_train, y_train)

# Get predictions for train
y_pred = model.predict(X_train)

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score
print('F1-Score: {}'.format(f1_score(y_train, y_pred, average='micro')))

cm = confusion_matrix(prop_caba_data[target].tolist(), y_pred)

# Creating a dataframe for a array-formatted Confusion matrix,so it will be easy for plotting.
cm_df = pd.DataFrame(cm,
                     index = ['Alto','Medio','Bajo'], 
                     columns = ['Alto','Medio','Bajo'])

#Plotting the confusion matrix
plt.figure(figsize=(10,8))
sns.heatmap(cm_df, cmap='Blues', annot=True, fmt='g')
plt.title('Confusion Matrix')
plt.ylabel('Actual Values')
plt.xlabel('Predicted Values')
plt.show()

Para train se obtiene un F1 cercano a 0.9, lo que en un principio resulta sospechoso, a continuación se analiza que sucede en test

In [ ]:
# Get predictions
y_pred = model.predict(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score
print('F1-Score: {}'.format(f1_score(y_test, y_pred, average='micro')))

cm = confusion_matrix(prop_caba_test[target].tolist(),y_pred)

# Creating a dataframe for a array-formatted Confusion matrix,so it will be easy for plotting.
cm_df = pd.DataFrame(cm,
                     index = ['Alto','Medio','Bajo'], 
                     columns = ['Alto','Medio','Bajo'])

#Plotting the confusion matrix
plt.figure(figsize=(10,8))
sns.heatmap(cm_df, cmap='Blues', annot=True, fmt='g')
plt.title('Confusion Matrix')
plt.ylabel('Actual Values')
plt.xlabel('Predicted Values')
plt.show()

Se obtuvo un F1 de aproximadamente 0.3 para test, mientras que en train se obtuvo F1 de 0.9, esto nos dice que el modelo performó bien en train pero muy mal en test, por lo que concluímos que el modelo hizo overfitting.

Se guarda el modelo Cat Boost entrenado con PCA

In [ ]:
import joblib  

try:
  joblib.dump(model, 'drive/My Drive/tp1 datos grupal/Modelos/CatBoostClassifier_PCA.pkl')
except:
  joblib.dump(model, 'CatBoostClassifier_PCA.pkl')

## Análisis de performance
En base a la métrica elegida para evaluar la performance de los modelos entrenados, es decir F1 score, se concluye que utilizando CatBoost Classifier se obtienen los mejores resultados, sin embargo la performance al entrenar CatBoost Classifier usando PCA no fue similar a la obtenida utilizando el DataSet original, sino que los resultados fueron bastante malos y el modelo overfitteo los datos.

Por otro lado, vimos que el arbol de decisión y el random forest obtuvieron métricas similares de f1 score a diferencia que el arbol de decisión se construyó con mayor cantidad de nodos pero el modelo de random forest tardó mayor tiempo en entrenarse.

#**4. Regresión**

In [ ]:

#diccionario para guardar performance de modelos para analisis
mse_model_performance = {}

## Regresión KNN

#### Grid search

In [ ]:
from sklearn import neighbors
#Conjunto de parámetros que quiero usar
params_grid = {'metric':['euclidean','manhattan'],
               'n_neighbors':list(range(2,6)),
               'weights': ['uniform','distance'], 
               'leaf_size':list(range(10,40,10))}
                
#Cantidad de folds utilizados
folds=3

#KFold
kfoldcv = KFold(n_splits=folds)

#Modelo
model = neighbors.KNeighborsRegressor()

#Usamos MSE como métrica para buscar hiperparámetros
scorer_fn = make_scorer(sk.metrics.mean_squared_error)

#GridSearch CV
gridcv = GridSearchCV(estimator=model,param_grid=params_grid,scoring=scorer_fn,cv=kfoldcv,return_train_score='True')

In [ ]:
gridcv.fit(x_train[:1000],train_target.values[:1000])

In [ ]:
#Mejores hiperparametros del arbol
print(gridcv.best_params_)
#Mejor métrica
print(gridcv.best_score_)

#### Mejor modelo

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


knn = neighbors.KNeighborsRegressor().set_params(**gridcv.best_params_)
knn.fit(x_train,train_target)

#### Evaluar 

In [ ]:
from sklearn import metrics

y_pred = knn.predict(x_train)
mse = metrics.mean_squared_error(
        y_true  = train_target,
        y_pred  = y_pred,
        squared = True
       )

print(f"El error (mse) de train es: {mse}")

y_pred = knn.predict(x_test)
mse = metrics.mean_squared_error(
        y_true  = test_target,
        y_pred  = y_pred,
        squared = True
       )

print(f"El error (mse) de test es: {mse}")
mse_model_performance['knn_regression'] = mse

In [ ]:
sns.scatterplot(x=test_target, y=y_pred,s=4)

In [ ]:
prop_caba_test.columns

In [ ]:
test_target.max()

In [ ]:
test_target.min()

Se guarda el modelo entrenado KNN:

In [ ]:
import joblib  

try:
  joblib.dump(knn, 'drive/My Drive/tp1 datos grupal/Modelos/RegresionKNN.pkl')
except:
  joblib.dump(knn, 'RegresionKNN.pkl')

### PCA

In [ ]:
#Separo un set de Evaluacion
y_train = df_PCA_ENCODED_train['property_price']
y_test = df_PCA_ENCODED_test['property_price']
X_train = df_PCA_ENCODED_train.drop('property_price', axis=1)
X_test = df_PCA_ENCODED_test.drop('property_price', axis=1)

#Verifico Cantidad de Datos en cada set
print('# Datos Entrenamiento: {}'.format(len(x_train)))
print('# Datos Prueba: {}'.format(len(x_test)))

In [ ]:
from sklearn import neighbors
#Conjunto de parámetros que quiero usar
params_grid = {'metric':['euclidean','manhattan'],
               'n_neighbors':list(range(2,6,2)),
               'weights': ['uniform','distance'], 
               'leaf_size':list(range(10,40,10))}
                
#Cantidad de folds utilizados
folds=3

#KFold
kfoldcv = KFold(n_splits=folds)

#Modelo
model = neighbors.KNeighborsRegressor()

#Usamos MSE como métrica para buscar hiperparámetros
scorer_fn = make_scorer(sk.metrics.mean_squared_error)

#GridSearch CV
gridcv = GridSearchCV(estimator=model,param_grid=params_grid,scoring=scorer_fn,cv=kfoldcv,return_train_score='True')

In [ ]:
gridcv.fit(X_train[:1000],y_train.values[:1000])

In [ ]:
knn = neighbors.KNeighborsRegressor().set_params(**gridcv.best_params_)
knn.fit(X_train,y_train)

In [ ]:
from sklearn import metrics

y_pred = knn.predict(X_train)
mse = metrics.mean_squared_error(
        y_true  = y_train,
        y_pred  = y_pred,
        squared = True
       )

print(f"El error (mse) de train es: {mse}")

y_pred = knn.predict(X_test)
mse = metrics.mean_squared_error(
        y_true  = y_test,
        y_pred  = y_pred,
        squared = True
       )

print(f"El error (mse) de test es: {mse}")
mse_model_performance['knn_regression_pca'] = mse

In [ ]:
sns.scatterplot(x=test_target, y=y_pred,s=4)

Se guarda el modelo KNN entrenado con PCA:

In [ ]:
import joblib  

try:
  joblib.dump(knn, 'drive/My Drive/tp1 datos grupal/Modelos/RegresionKNN_PCA.pkl')
except:
  joblib.dump(knn, 'RegresionKNN_PCA.pkl')

### Análisis de performance
La calidad del modelo KNN es la más baja, con muchos puntos muy alejados del valor real. 

TODO: razonar por qué predijo mal, lo que peor predijo

## XGBoost

In [ ]:
from sklearn.metrics import mean_squared_error
import xgboost as xgb

#### Grid search

In [ ]:
#Conjunto de parámetros que quiero usar
params_grid = {"learning_rate": (0.1, 0.2, 0.4),
                        "max_depth": [8, 16, 32],
                        "min_child_weight": [5, 10, 20],
                        "gamma":[ 0.0, 0.2, 0.4],
                        "colsample_bytree":[ 0.3, 0.4, 0.5]}
                
#Cantidad de folds utilizados
folds=3

#KFold
kfoldcv = KFold(n_splits=folds)

#Modelo
model =  xgb.XGBRegressor(objective = "reg:squarederror")

#Usamos MSE como métrica para buscar hiperparámetros
scorer_fn = make_scorer(sk.metrics.mean_squared_error)

#GridSearch CV
gridcv = GridSearchCV(estimator=model,param_grid=params_grid,scoring=scorer_fn,cv=kfoldcv,return_train_score='True')

In [ ]:
gridcv.fit(x_train[:100],train_target.values[:100])

In [ ]:
#Mejores hiperparametros del arbol
print(gridcv.best_params_)
#Mejor métrica
print(gridcv.best_score_)

#### Mejor modelo

In [ ]:
reg =  xgb.XGBRegressor(**gridcv.best_params_)
reg.fit(x_train,train_target)

In [ ]:
from sklearn import metrics

y_pred = reg.predict(x_train)
mse = metrics.mean_squared_error(
        y_true  = train_target,
        y_pred  = y_pred,
        squared = True
       )

print(f"El error (mse) de train es: {mse}")

y_pred = reg.predict(x_test)
mse = metrics.mean_squared_error(
        y_true  = test_target,
        y_pred  = y_pred,
        squared = True
       )

print(f"El error (mse) de test es: {mse}")
mse_model_performance['xgb'] = mse

In [ ]:
sns.scatterplot(x=test_target, y=y_pred,s=4)

Se guarda el modelo XGBoost entrenado:

In [ ]:
import joblib  

try:
  joblib.dump(reg, 'drive/My Drive/tp1 datos grupal/Modelos/RegresionXGBoost.pkl')
except:
  joblib.dump(reg, 'RegresionXGBoost.pkl')

### PCA

In [ ]:
#Separo un set de Evaluacion
y_train = df_PCA_ENCODED_train['property_price']
y_test = df_PCA_ENCODED_test['property_price']
X_train = df_PCA_ENCODED_train.drop('property_price', axis=1)
X_test = df_PCA_ENCODED_test.drop('property_price', axis=1)

#Verifico Cantidad de Datos en cada set
print('# Datos Entrenamiento: {}'.format(len(x_train)))
print('# Datos Prueba: {}'.format(len(x_test)))

In [ ]:
#Conjunto de parámetros que quiero usar
params_grid = {"learning_rate": (0.1, 0.2, 0.4),
                        "max_depth": [8, 16, 32],
                        "min_child_weight": [5, 10, 20],
                        "gamma":[ 0.0, 0.2, 0.4],
                        "colsample_bytree":[ 0.3, 0.4, 0.5]}
                
#Cantidad de folds utilizados
folds=3

#KFold
kfoldcv = KFold(n_splits=folds)

#Modelo
model =  xgb.XGBRegressor(objective = "reg:squarederror")

#Usamos MSE como métrica para buscar hiperparámetros
scorer_fn = make_scorer(sk.metrics.mean_squared_error)

#GridSearch CV
gridcv = GridSearchCV(estimator=model,param_grid=params_grid,scoring=scorer_fn,cv=kfoldcv,return_train_score='True')

In [ ]:
gridcv.fit(X_train[:100],y_train.values[:100])

In [ ]:
#Mejores hiperparametros del arbol
print(gridcv.best_params_)
#Mejor métrica
print(gridcv.best_score_)

In [ ]:
reg =  xgb.XGBRegressor(**gridcv.best_params_)
reg.fit(X_train,y_train)

In [ ]:
from sklearn import metrics

y_pred = reg.predict(X_train)
mse = metrics.mean_squared_error(
        y_true  = y_train,
        y_pred  = y_pred,
        squared = True
       )

print(f"El error (mse) de train es: {mse}")

y_pred = reg.predict(X_test)
mse = metrics.mean_squared_error(
        y_true  = y_test,
        y_pred  = y_pred,
        squared = True
       )

print(f"El error (mse) de test es: {mse}")
mse_model_performance['xgb_pca'] = mse

In [ ]:
sns.scatterplot(x=test_target, y=y_pred,s=4)

### Análisis de performance
El error MSE es proporcional a la escala de los precios de las propiedades. Aun así, se ve del gráfico que hay una amplia diferencia entre los valores reales y predichos, a menudo de hasta el doble del valor real. La performance del modelo es mediocre. Sin embargo, tiene una performance en la misma magnitud en train y test. Usando PCA se obtuvieron resultados similares.

Se guarda el modelo XGBoost entrenado con PCA:

In [ ]:
import joblib  

try:
  joblib.dump(reg, 'drive/My Drive/tp1 datos grupal/Modelos/RegresionXGBoost_PCA.pkl')
except:
  joblib.dump(reg, 'RegresionXGBoost_PCA.pkl')

## CatBoost Regressor

### Dataset original

In [ ]:
!pip install catboost
import catboost as cb

In [ ]:
import numpy as np

model =  cb.CatBoostRegressor()

grid = {'learning_rate': [0.7, 0.8, 0.9],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1,10,20]}

grid_search_result = model.grid_search(grid, 
                                       X=x_train[:100], 
                                       y=train_target.values[:100], verbose = False)

In [ ]:
grid_search_result['params']

In [ ]:
model = cb.CatBoostRegressor(l2_leaf_reg = grid_search_result['params']['l2_leaf_reg'],
                          learning_rate=grid_search_result['params']['learning_rate'],
                          depth=grid_search_result['params']['depth'], verbose = False)
# Fit model
model.fit(x_train, train_target)
# Get predictions for train
preds = model.predict(x_train)

In [ ]:
mse = metrics.mean_squared_error(
        y_true  = train_target,
        y_pred  = preds,
        squared = True
       )

print(f"El error (mse) de train es: {mse}")

In [ ]:
# Get predictions
preds = model.predict(x_test)

In [ ]:
mse = metrics.mean_squared_error(
        y_true  = test_target,
        y_pred  = preds,
        squared = True
       )

print(f"El error (mse) de test es: {mse}")
mse_model_performance['cat_regression'] = mse

In [ ]:
sns.scatterplot(x=test_target, y=preds,  s=4)

Se guarda el modelo Cat Boost

In [ ]:
import joblib  

try:
  joblib.dump(model, 'drive/My Drive/tp1 datos grupal/Modelos/RegresionCatBoost.pkl')
except:
  joblib.dump(model, 'RegresionCatBoost.pkl')

### PCA

In [ ]:
#Separo un set de Evaluacion
y_train = df_PCA_ENCODED_train['property_price']
y_test = df_PCA_ENCODED_test['property_price']
X_train = df_PCA_ENCODED_train.drop('property_price', axis=1)
X_test = df_PCA_ENCODED_test.drop('property_price', axis=1)

#Verifico Cantidad de Datos en cada set
print('# Datos Entrenamiento: {}'.format(len(x_train)))
print('# Datos Prueba: {}'.format(len(x_test)))


In [ ]:
model =   cb.CatBoostRegressor()

grid = {'learning_rate': [0.7, 0.8, 0.9],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1,10,20]}

grid_search_result = model.grid_search(grid, 
                                       X=X_train[:100], 
                                       y=y_train.values[:100])

In [ ]:
model = cb.CatBoostRegressor(l2_leaf_reg = grid_search_result['params']['l2_leaf_reg'],
                          learning_rate=grid_search_result['params']['learning_rate'],
                          depth=grid_search_result['params']['depth'])
# Fit model
model.fit(X_train, y_train)
# Get predictions for train
preds = model.predict(X_train)

In [ ]:
mse = metrics.mean_squared_error(
        y_true  = y_train,
        y_pred  = preds,
        squared = True
       )

print(f"El error (mse) de train es: {mse}")

In [ ]:
# Get predictions
preds = model.predict(X_test)

In [ ]:
sns.scatterplot(x=y_test, y=preds,  s=4)

In [ ]:
mse = metrics.mean_squared_error(
        y_true  = y_test,
        y_pred  = preds,
        squared = True
       )

print(f"El error (mse) de test es: {mse}")
mse_model_performance['cat_regression_pca'] = mse

Se guarda el modelo Cat Boost entrenado con PCA

In [ ]:
import joblib  

try:
  joblib.dump(model, 'drive/My Drive/tp1 datos grupal/Modelos/RegresionCatBoost_PCA.pkl')
except:
  joblib.dump(model, 'RegresionCatBoost_PCA.pkl')

### Análisis de performance
Comparamos el error MSE de cada modelo:

In [ ]:
pairs = mse_model_performance.items()
keys = list(map(lambda pair: pair[0], pairs))
values = list(map(lambda pair: pair[1], pairs))

In [ ]:
sns.barplot(x = keys, y=values)

Se ve que la mejor performance la lograron XGB y CatBoost Regression, sin usar el dataset PCA.

In [ ]:
print("Performance de xgb:" + str(mse_model_performance['xgb']))
print("Performance de cat boost:" + str(mse_model_performance['cat_regression']))

#**5. Conclusiones**

A lo largo del desarrollo del TP se aplicaron diversas técnicas aprendidas en la teoría, pero que se terminaron de afianzar y entender en la práctica.

Nos hubiera gustado explorar más técnicas de preprocesamiento de datos, creemos que identificar mejor casos especiales de propiedades, datos mal ingresados como coordenadas incorrectas, dónde codificar y transformar u omitir ciertas variables habría mejorado mucho la performance en modelos individuales. Además, no se tuvo en cuenta el dataset con las descripciones de las propiedades, probablemente se podrían haber extraído conclusiones y mejorado ciertos resultados a partir de ello, pero consideramos que excedió el alcance de este TP.

Los modelos entrenados en general performaron de buena forma. Se pudo poner a prueba las ventajas y desventajas de cada uno de ellos, destacando CatBoost, que comprobamos que tiene una potencia muy interesante pero tiene como desventaja por ejemplo la búsqueda de hiperparámetros y el tiempo de ejecución.

 En cuanto a regresión, se logró obtener modelos que están correlacionados a los valores reales en testing, obteniendo mejores resultados con árboles que en la regresión KNN.

En cuanto a clasificación, se intentó en un principio entrenar el modelo a elección utilizando Support Vector Machine, más precisamente Support Vector Classifier, pero no dió sus frutos debido a que la busqueda de hiperparámetros se hizo imposible. Por lo que se decidió entrenar un modelo utilizando Cat Boost, y resultó ser nuestro mejor modelo para clasificar las propiedades según su 'tipo_precio'.

Las clases teóricas y prácticas resultaron de mucha ayuda para la realización del TP, en especial las notebooks de la práctica, que muchas veces ahorran el trabajo de empezar algo desde cero.